In [1]:
import wandb
from data_loaders import make_train_data, make_test_val_data, make_model
from train_fed import train, test
import torch
import time
import gc
import lstm
import config_file
from clean_data import load_all_houses_with_device
import random
import optuna
from data_loaders import PecanStreetDataset
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import numpy as np
from lstm import LSTM
from clean_data import normalize_y
from sklearn import metrics

In [2]:
wandb.login()

print(torch.cuda.is_available())

gc.collect()
torch.cuda.empty_cache()

print(torch.__version__)


config_ = config_file.load_hyperparameters("refrigerator1")

wandb: Currently logged in as: nilm (use `wandb login --relogin` to force relogin)


True
1.10.2


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
homes = load_all_houses_with_device(config_file.path, config_['appliance'])

In [5]:
def client_update(client_model, optimizer, train_loader, epoch=config_['epochs']):
    model.train()
    for e in range(epoch):
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.cuda(), target.cuda()
            optimizer.zero_grad()
            output = client_model(data)
            loss = nn.MSELoss(output, target)
            loss.backward()
            optimizer.step()
    return float(loss.item())

In [6]:
def server_aggregate(global_model, client_models):
    global_dict = global_model.state_dict()
    for k in global_dict.keys():
        global_dict[k] = torch.stack([client_models[i].state_dict()[k] for i in range(len(client_models))], 0).mean(0)
    global_model.load_state_dict(global_dict)
    for model in client_models:
        model.load_state_dict(global_model.state_dict())

In [7]:
def model_pipeline(hyperparameters, train_months, test_month, appliance, window_length, train_buildings,
                   test_buildings, patience):
    with wandb.init(project="jan27_FL_trials", config=hyperparameters):
        wandb.run.name = str(config_['appliance'])+"_Test:"+str(test_buildings)+"_Train:" + str(train_buildings)

        config = wandb.config

        # lengths = [85320, 132480, 132480, 132480, 132480, 132480, 132480]

        global_model, criterion, optimizer = make_model(config)
        
        client_models = [lstm.LSTM(
            config.in_channels,
            config.out_channels,
            config.kernel_size,
            config.hidden_size_1,
            config.hidden_size_2,
            config.fc1
        ).to(device) for _ in range(len(train_buildings))]
        
        for model in client_models:
            model.load_state_dict(global_model.state_dict())
            
        optimizers = [torch.optim.AdamW(
            model.parameters(),
            lr = config.learning_rate,
            weight_decay = config.weight_decay
        ) for model in client_models]
        
        print(global_model)
        print("Window Length: ", window_length)
        
        wandb.watch(global_model, criterion, log="all", log_freq=10)

        validation_loader, test_loader, test_val_seq_min, test_val_seq_max  = make_test_val_data(
            config,
            test_month,
            appliance,
            window_length,
            test_buildings
        )
        
        test_results = []
        train_results = []
        example_ct = 0
        batch_ct = 0
        all_epochs = 0
        for r in range(20):
            client_losses = 0.0
            gc.collect()
            torch.cuda.empty_cache()
            for i in range(len(train_buildings)):
                #wandb.watch(client_models[i], criterion, log="all", log_freq=10)
                time_log = time.time()
                #scheduler = torch.optim.lr_scheduler.StepLR(optimizers[i], step_size=50, gamma=0.9, verbose=True)
                scheduler = torch.optim.lr_scheduler.CyclicLR(
                    optimizers[i],
                    base_lr = 0.1*config_['learning_rate'],
                    max_lr = 5*config_['learning_rate'],
                    step_size_up = 50,
                    step_size_down = 6000,
                    gamma = 1,
                    cycle_momentum=False,
                    verbose=False
                )
                
                train_loader, train_seq_min, train_seq_max = make_train_data(
                    config,
                    train_months,
                    appliance,
                    window_length,
                    [train_buildings[i]]
                )
                
                #loss += client_update(client_models[i], optimizer[i], train_loader, epochs = config.epochs)
                example_ct, batch_ct, all_epochs, _, client_train_loss = train(
                    client_models[i].to(device),
                    train_loader,
                    validation_loader,
                    criterion,
                    optimizers[i],
                    config,
                    example_ct,
                    batch_ct,
                    all_epochs,
                    scheduler,
                    test_val_seq_min,
                    test_val_seq_max,
                    train_seq_min,
                    train_seq_max,
                    patience
                )
                
                client_losses += client_train_loss
                
                print("Time to train on one home: ", time.time() - time_log)
            
            client_losses = client_losses/len(train_buildings)
            server_aggregate(global_model, client_models)
            test_results.append(test(global_model, test_loader, criterion, test_val_seq_min, test_val_seq_max))
            train_results.append(client_losses)
            
            print("train_results: ", train_results)
            print("test_results: ", test_results)
            print("Round_"+str(r)+"_results: ", test(global_model, test_loader, criterion, test_val_seq_min, test_val_seq_max))

    return train_results, test_results, global_model

In [8]:
home_ids = homes.dataid.unique()

In [10]:
len(home_ids)

33

In [11]:
random_select = [5,10,15,20,25,30,33]

In [14]:
final_results = {}
random.seed(3)
global_models = []

for i in range(1):
    gc.collect()
    torch.cuda.empty_cache()
    #train_homes = home_ids
    train_homes = random.sample(list(home_ids), k=random_select[-1])
    test_homes = [random.choice(train_homes)]
    patience = 25
    print("patience: ", patience)
    print("training_home: ", train_homes)
    print("test_home: ", test_homes)
    train_results, test_results, global_model = model_pipeline(
    config_,
    'sept_oct_nov',
    'dec',
    config_['appliance'],
    config_['window_size'],
    train_homes,
    test_homes,
    patience)
    global_models.append(global_model)
    result = {"Train_home_"+str(train_homes)+"_Test_home_"+str(test_homes): test_results}
    final_results.update(result)
    print(final_results)
    #torch.save(model.state_dict(), r"C:\Users\aar245\Desktop\FL_models")

patience:  25
training_home:  [5058, 2358, 3488, 10182, 6672, 11878, 7021, 6526, 335, 10164, 142, 9973, 10983, 6240, 2126, 1417, 7069, 6178, 8825, 3383, 9004, 387, 3976, 3700, 9290, 3996, 690, 7365, 9053, 183, 2561, 526, 10811]
test_home:  [2358]


wandb: wandb version 0.12.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


LSTM(
  (conv1): Conv1d(1, 16, kernel_size=(7,), stride=(1,), padding=(3,))
  (lstm1): LSTM(16, 64, batch_first=True, bidirectional=True)
  (lstm2): LSTM(128, 85, batch_first=True, bidirectional=True)
  (attention): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=170, out_features=170, bias=True)
  )
  (linear1): Linear(in_features=170, out_features=211, bias=True)
  (linear2): Linear(in_features=211, out_features=1, bias=True)
  (relu): ReLU()
  (leaky): LeakyReLU(negative_slope=0.01)
  (maxpool): MaxPool1d(kernel_size=5, stride=1, padding=2, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.5, inplace=False)
)
Window Length:  136
trigger times: 0
Loss after 00963 batches: 0.1937
trigger times: 1
Loss after 01926 batches: 0.1688
trigger times: 2
Loss after 02889 batches: 0.1155
trigger times: 3
Loss after 03852 batches: 0.1025
trigger times: 4
Loss after 04815 batches: 0.0992
trigger times: 5
Loss after 05778 batches: 0.0972
trigger times: 6
Loss af

trigger times: 4
Loss after 137409 batches: 0.0646
trigger times: 5
Loss after 138372 batches: 0.0583
trigger times: 6
Loss after 139335 batches: 0.0570
trigger times: 7
Loss after 140298 batches: 0.0556
trigger times: 8
Loss after 141261 batches: 0.0554
trigger times: 9
Loss after 142224 batches: 0.0548
trigger times: 10
Loss after 143187 batches: 0.0548
trigger times: 11
Loss after 144150 batches: 0.0547
trigger times: 12
Loss after 145113 batches: 0.0545
trigger times: 13
Loss after 146076 batches: 0.0545
trigger times: 14
Loss after 147039 batches: 0.0544
trigger times: 15
Loss after 148002 batches: 0.0543
trigger times: 16
Loss after 148965 batches: 0.0542
trigger times: 17
Loss after 149928 batches: 0.0542
trigger times: 18
Loss after 150891 batches: 0.0542
trigger times: 19
Loss after 151854 batches: 0.0541
trigger times: 20
Loss after 152817 batches: 0.0543
trigger times: 21
Loss after 153780 batches: 0.0538
trigger times: 22
Loss after 154743 batches: 0.0540
trigger times: 23


C:\Users\aar245\Anaconda3\envs\privacy_ml_env\lib\site-packages\numpy\core\_methods.py:233: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


...

C:\Users\aar245\Anaconda3\envs\privacy_ml_env\lib\site-packages\numpy\core\_methods.py:233: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


trigger times: 0
Loss after 21079439 batches: 0.0884
trigger times: 1
Loss after 21080402 batches: 0.0749
trigger times: 2
Loss after 21081365 batches: 0.0734
trigger times: 3
Loss after 21082328 batches: 0.0692
trigger times: 4
Loss after 21083291 batches: 0.0653
trigger times: 5
Loss after 21084254 batches: 0.0613
trigger times: 6
Loss after 21085217 batches: 0.0581
trigger times: 7
Loss after 21086180 batches: 0.0536
trigger times: 8
Loss after 21087143 batches: 0.0516
trigger times: 9
Loss after 21088106 batches: 0.0527
trigger times: 10
Loss after 21089069 batches: 0.0486
trigger times: 11
Loss after 21090032 batches: 0.0493
trigger times: 12
Loss after 21090995 batches: 0.0471
trigger times: 13
Loss after 21091958 batches: 0.0471
trigger times: 14
Loss after 21092921 batches: 0.0479
trigger times: 15
Loss after 21093884 batches: 0.0464
trigger times: 16
Loss after 21094847 batches: 0.0462
trigger times: 17
Loss after 21095810 batches: 0.0442
trigger times: 18
Loss after 21096773 

Round_19_results:  [0.13793548941612244, 0.14146804396514878, 0.32086843455623654, 1.0714886450573733, 0.6659048185629647, 35.39220885338468, 3238.1082]


Test_F1_Score,▁▁▄▄▇▇▆▆▇▇▇▇██▇▇██████████████████████▇▇
Test_Loss,▇▇██▃▃▆▆▂▂▅▅▂▂▆▆▁▁▇▇▁▁▆▆▁▁▆▆▁▁▆▆▁▁▆▆▁▁▇▇
Test_MAE,▇▇██▄▄▅▅▂▂▅▅▁▁▅▅▁▁▅▅▁▁▅▅▁▁▅▅▁▁▅▅▁▁▅▅▁▁▆▆
Test_MSE,██▅▅▂▂▃▃▂▂▂▂▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂
Test_NDE,██▅▅▂▂▃▃▂▂▂▂▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂
Test_NEP,▇▇██▄▄▅▅▂▂▅▅▁▁▅▅▁▁▅▅▁▁▅▅▁▁▅▅▁▁▅▅▁▁▅▅▁▁▆▆
Test_R2_Value,▁▁▄▄▇▇▆▆▇▇▇▇██▇▇██████████████████████▇▇
Training_F1,▂▃▂▂▁█▃▃▅▅▇▅▄▆▇▅▆▄▅▄█▄▅▄▇▅▄▅▅▃▅▇▅▄▇▄▆▆▇▆
Training_Loss,█▇▄▅▆▁▄▅▄▃▂▃▅▂▂▃▄▆▃▃▁▆▃▄▂▄▅▄▃▅▂▂▃▄▃▄▃▃▂▃
Training_MAE,█▅▅▄▆▁▅▆▃▃▂▄▆▂▂▄▄▆▂▄▁▆▂▄▃▁▄▁▃▆▃▂▃▆▃▄▂▃▂▂
Training_MSE,▆▃▄▃▅▁▆▅▂▃▂▅▄▂▂▅▃▅▂▄▁▅▂▄▃▁▃▁█▄▇▂█▄▃▂▂▄▂▂


{'Train_home_[5058, 2358, 3488, 10182, 6672, 11878, 7021, 6526, 335, 10164, 142, 9973, 10983, 6240, 2126, 1417, 7069, 6178, 8825, 3383, 9004, 387, 3976, 3700, 9290, 3996, 690, 7365, 9053, 183, 2561, 526, 10811]_Test_home_[2358]': [[0.1372801512479782, -0.17630011535437906, 0.09165623225234511, 1.1074626263238363, 0.9123759614671366, 36.58046097732784, 4436.628], [0.15019331872463226, -0.03945367500616581, 0.18360298702085523, 1.1433348997800294, 0.8062334854689305, 37.76535360317488, 3920.487], [0.10503555089235306, 0.11604646509476468, 0.2941523892332113, 1.0160329939295982, 0.6856226066458603, 33.56046010283253, 3333.9902], [0.12885217368602753, 0.08478938334622399, 0.2717248960955158, 1.0659429087240713, 0.709866598204025, 35.209028322765505, 3451.882], [0.0979808047413826, 0.13873112342958427, 0.31531024815199815, 0.9756221648036203, 0.6680276622646856, 32.225655006238654, 3248.4307], [0.12634874880313873, 0.12620499423095177, 0.3036101279672065, 1.0441931429634264, 0.6777433262115

In [ ]:
#FL Results for 2 buildings and 7 steps of 5 buildings
train_results:  [0.014415281222734066, 0.013136782105790362, 0.010528174418556902, 0.00805639523009415, 0.010109599870269274, 0.005883289906584347, 0.005446426593005612]
test_results:  [[0.069185771048069, -0.11199728930570707, 0.32316803403925415, 0.9517108260147541, 0.7416519323543487, 40.07046553667854, 3947.6233], [0.05734777823090553, 0.06975431141155508, 0.44407010791422613, 0.832600811550101, 0.6204318268877594, 35.05550342926507, 3302.3997], [0.037569355219602585, 0.1251698633599584, 0.49302403189904215, 0.7293787760707009, 0.583472149127272, 30.709482660936658, 3105.6729], [0.04163476824760437, 0.18916286313756836, 0.5210445544397486, 0.7227430920088246, 0.5407917210373157, 30.430096378627432, 2878.496], [0.043641287833452225, 0.24672802107603853, 0.5520633916122849, 0.7305134714355811, 0.5023983660510822, 30.757257436919247, 2674.1377], [0.03620066121220589, 0.24254624570391647, 0.5397654893540841, 0.7007306168059849, 0.5051874079722667, 29.503291613060227, 2688.9834], [0.036571137607097626, 0.22816064587399654, 0.5304912918947414, 0.7137260700904927, 0.5147819631240661, 30.050447165709485, 2740.0522]]
Round_6_results:  [0.036571137607097626, 0.22816064587399654, 0.5304912918947414, 0.7137260700904927, 0.5147819631240661, 30.050447165709485, 2740.0522]

In [ ]:
#FL Results for 33 buildings
{'Train_home_[2126, 6240, 3700, 6672, 7021, 335, 142, 9973, 2358, 2126, 11878, 5058, 9973, 5058, 7069, 526, 7021, 10164, 6240, 9004, 7365, 335, 9053, 6672, 2561, 183, 10164, 5058, 8825, 10182, 8825, 10811, 3976]_Test_home_[8825]': [[0.10082405805587769, -0.054764222263741, 0.1231860661470977, 1.1374510211962554, 0.9493533524329186, 36.445046007547695, 9752.355], [0.11666058003902435, 0.0033553729248713138, 0.22448578730774937, 1.1967830488395308, 0.8970421050311045, 38.34610234921426, 9214.982], [0.09502508491277695, 0.06306210793407208, 0.4091371338812714, 1.0743021801538923, 0.8433023433560337, 34.421695222129436, 8662.934], [0.09201239049434662, 0.0888807369859389, 0.4750409455392341, 0.9362492796710058, 0.8200639469571572, 29.998344927641377, 8424.214], [0.06690056622028351, 0.11779342491118161, 0.575479130561629, 0.7734026148743284, 0.7940407462593525, 24.780578113867872, 8156.887], [0.08562850952148438, 0.09116170711364147, 0.5687260656187799, 0.9169960265281519, 0.8180109409907265, 29.381451818830143, 8403.124], [0.0629526749253273, 0.11004705945059767, 0.5942846983018615, 0.8012983318796003, 0.8010129671558082, 25.674384238901233, 8228.51], [0.0780257135629654, 0.1035195067392124, 0.5885601973830207, 0.8781840137793461, 0.806888160052532, 28.13787687457617, 8288.863], [0.06482576578855515, 0.11552502542995946, 0.6041126568254397, 0.7994763119207765, 0.796082468981858, 25.61600493290161, 8177.861], [0.0757279247045517, 0.10266384027132536, 0.5909740425258612, 0.8803234656976516, 0.8076583271010261, 28.206427011804568, 8296.775], [0.06181953847408295, 0.10445801892372542, 0.5961459732150404, 0.831618823847048, 0.806043429565136, 26.645882531252276, 8280.187], [0.07333928346633911, 0.09887557787656209, 0.5904951093552152, 0.8805827319948272, 0.8110679704670578, 28.214734158180708, 8331.802], [0.06249981373548508, 0.1056225345966233, 0.6023961404422763, 0.8395811685763604, 0.8049953091236983, 26.90100386358231, 8269.419], [0.07111044973134995, 0.11364462231541128, 0.6015437535926366, 0.8360909977276793, 0.7977749276509928, 26.78917536742376, 8195.247], [0.06206701323390007, 0.09726051170449412, 0.5975866108942892, 0.8595424626232535, 0.8125216526577462, 27.540583297204083, 8346.734], [0.07088565826416016, 0.11302232012409985, 0.6007153971593981, 0.8537924311344753, 0.7983350331303676, 27.35634665030831, 8201.001], [0.06324874609708786, 0.09630308423125122, 0.5977169726517698, 0.8716301579538709, 0.8133833873815131, 27.927884907770363, 8355.587], [0.07241006940603256, 0.1018741345383698, 0.5940447527818379, 0.8890966213663465, 0.8083691035743891, 28.487527521645152, 8304.077], [0.06157456710934639, 0.09768669531039154, 0.5995392137772617, 0.860373560964417, 0.8121380691527804, 27.567212502959812, 8342.794], [0.06892590969800949, 0.10954055344572677, 0.5985059514769252, 0.8515056738163342, 0.8014688307189015, 27.28307670363395, 8233.193]]}

In [15]:
global_models[0]

LSTM(
  (conv1): Conv1d(1, 16, kernel_size=(7,), stride=(1,), padding=(3,))
  (lstm1): LSTM(16, 64, batch_first=True, bidirectional=True)
  (lstm2): LSTM(128, 85, batch_first=True, bidirectional=True)
  (attention): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=170, out_features=170, bias=True)
  )
  (linear1): Linear(in_features=170, out_features=211, bias=True)
  (linear2): Linear(in_features=211, out_features=1, bias=True)
  (relu): ReLU()
  (leaky): LeakyReLU(negative_slope=0.01)
  (maxpool): MaxPool1d(kernel_size=5, stride=1, padding=2, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [16]:
global_models[0].cpu()

LSTM(
  (conv1): Conv1d(1, 16, kernel_size=(7,), stride=(1,), padding=(3,))
  (lstm1): LSTM(16, 64, batch_first=True, bidirectional=True)
  (lstm2): LSTM(128, 85, batch_first=True, bidirectional=True)
  (attention): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=170, out_features=170, bias=True)
  )
  (linear1): Linear(in_features=170, out_features=211, bias=True)
  (linear2): Linear(in_features=211, out_features=1, bias=True)
  (relu): ReLU()
  (leaky): LeakyReLU(negative_slope=0.01)
  (maxpool): MaxPool1d(kernel_size=5, stride=1, padding=2, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [17]:
PATH = r"C:\Users\aar245\Desktop\privacy_preserving_nn\models"

In [18]:
torch.save(global_models[0].state_dict(), PATH+"\\FL_refrigerator_model_kmeans_group0_33_homes_new.pth")

In [ ]:
model_test = LSTM(
        config_['in_channels'],
        config_['out_channels'],
        config_['kernel_size'],
        config_['hidden_size_1'],
        config_['hidden_size_2'],
        config_['fc1'])

In [ ]:
model_test.load_state_dict(torch.load(PATH+"\\FL_trial_1_refrigerator_model_kmeans_group_0.pth"))
model_test.eval()

In [ ]:
def load_model(appliance, num_homes, trial):
    model_test = LSTM(
        config_['in_channels'],
        config_['out_channels'],
        config_['kernel_size'],
        config_['hidden_size_1'],
        config_['hidden_size_2'],
        config_['fc1'])
    
    root_path = r"C:\Users\aar245.CORNELL\Desktop\privacy_preserving_nn\models\\"
    
    model_test.load_state_dict(torch.load(root_path+str(appliance)+"_model_total_houses_"+str(num_homes)+"_trial_"+str(trial)+".pth"))
    model_test.eval()
    
    return model_test

In [ ]:
def load_test_data_loader(appliance, test_home, test_month):    
    test_path = r"C:\Users\aar245.CORNELL\Desktop\privacy_preserving_nn\input\1min_real_"+str(test_month)+"2019.csv"
    test_dataset = PecanStreetDataset(test_path, str(appliance)+"1", config_['window_size'], [test_home])
    test_loader = DataLoader(dataset=test_dataset, batch_size=config_['batch_size'], shuffle = False, num_workers=0)
    y_min = test_dataset.y_min
    y_max = test_dataset.y_max
    
    return test_loader, y_min, y_max

In [ ]:
def create_true_and_pred_lists(model_test, test_loader, y_min, y_max):
    #model_test = load_model(appliance, num_homes, trial)
    predictions = []
    true_vals = []
    for i, (inputs, outputs) in enumerate(test_loader):
        inputs, outputs = inputs.cpu(), outputs.cpu()
        prediction = model_test(inputs)
        predictions.append(prediction.detach().numpy())
        true_vals.append(outputs.detach().numpy())
    
    true_vals = [i for subitems in true_vals for i in subitems]
    true_vals = [i for subitems in true_vals for i in subitems]
    true_vals = denormalize(true_vals, y_min, y_max)
    
    predictions = [i for subitems in predictions for i in subitems]
    predictions = [i for subitems in predictions for i in subitems]
    predictions = denormalize(predictions, y_min, y_max)
    
    return true_vals, predictions

In [ ]:
def denormalize(sequence, y_min, y_max):
    sequence = np.add(np.multiply(sequence, y_max - y_min), y_min)
    sequence = np.exp(sequence) - 1
    return sequence

In [ ]:
def plot_true_and_pred(true, predictions, view):
    if view == 0:
        plt.figure(figsize=(20,4))
        plt.plot(true)
        plt.plot(predictions, alpha = 0.7)
        plt.plot([np.mean(true)]*len(true))
        plt.legend(['true', 'pred', 'mean'])
        
    elif view > 0:
        plt.figure(figsize=(20,4))
        plt.plot(true[0:view])
        plt.plot(predictions[0:view], alpha = 0.7)
        plt.plot([np.mean(true)]*view)
        plt.legend(['true', 'pred', 'mean'])

In [ ]:
def calculate_model_metrics(true, predictions, y_min, y_max):
    local_precision = precision(predictions, true)
    local_recall = recall(predictions, true)

    mse = np.mean(np.square(predictions - true))
    r2 = metrics.r2_score(predictions, true)
    f1 = f1_score(local_precision, local_recall)
    nep = np.sum(abs(predictions - true))/np.sum(true)
    nde = np.sum(np.square(predictions - true))/np.sum(np.square(true))
    mae = np.sum(abs(predictions - true))/len(true)
    
    results = {"MSE":mse, "R2":r2, "F1":f1, "NEP":nep, "NDE":nde, "MAE":mae}
    return results

In [ ]:
test_loader, y_min, y_max = load_test_data_loader("refrigerator", 3996, "dec")
true, pred = create_true_and_pred_lists(model_test, test_loader, y_min, y_max)
plot_true_and_pred(true, pred, 0)

In [ ]:
for home in home_ids:
    test_loader, y_min, y_max = load_test_data_loader("refrigerator", home, "dec")
    true, pred = create_true_and_pred_lists(model_test, test_loader, y_min, y_max)
    plot_true_and_pred(true, pred, 4000)

In [ ]:
#Results from FL trial 1: Only train on homes from kmeans group 0
train_results:  [0.05918259809798596, 0.05631256998604834, 0.05440947504993626, 0.05202335131766571, 0.0503139231996005, 0.048780418103246403, 0.047837128240590214, 0.04677401642300567, 0.04564181936058709, 0.04503467424225199, 0.044160787837709645, 0.04364045831778884, 0.04269331134775909, 0.042177348902872995, 0.04186163381211489, 0.04121293036455554, 0.04038131841834548, 0.040097079220779215, 0.03981643070223546, 0.039571967459759584]
test_results:  [[0.10141171514987946, 0.0037996456822343427, 0.30775005794404453, 1.1624964525961219, 0.7102131519348384, 32.36117409223122], [0.07018595188856125, -0.03035194696439647, 0.29275949364898596, 1.0193813194259513, 0.7345605631401784, 28.37718452442667], [0.07101921737194061, 0.024487967353850726, 0.33216459819150085, 0.9911870298545128, 0.6954639814554395, 27.592321644896586], [0.0762869194149971, 0.037722537982371684, 0.3469836294700992, 1.0001555309118728, 0.6860287619779427, 27.8419836747595], [0.07988928258419037, 0.046875361692992, 0.3580031460561334, 1.007971290365539, 0.6795035147228619, 28.059556082639325], [0.08263445645570755, 0.06088193306524636, 0.3722769571476962, 1.0117267310898002, 0.6695179288378231, 28.164098742360537], [0.08711171895265579, 0.0651336946658907, 0.3775209821011063, 1.0278720855497583, 0.6664867473821405, 28.61354753447742], [0.09034138172864914, 0.058741536735728794, 0.3788472553128205, 1.0409449550909322, 0.6710438600997121, 28.977465554323565], [0.09514638781547546, 0.047170316993679906, 0.37581896125501857, 1.063129654702191, 0.6792932333429572, 29.595035547505493], [0.09916041046380997, 0.04520241126403168, 0.3788573920789103, 1.0829578383335274, 0.6806961968598882, 30.147005664053744], [0.09965900331735611, 0.0475078473168451, 0.38485057828841956, 1.083835660609424, 0.6790526045305888, 30.17144217689547], [0.09750136733055115, 0.06263643169646227, 0.38969362416454856, 1.0732437942194344, 0.6682671053889663, 29.876589464491374], [0.10068412125110626, 0.07135716647367518, 0.40400621237323964, 1.0842488674066784, 0.6620498952202254, 30.18294488477228], [0.10021708905696869, 0.07195186144252641, 0.4020264338053921, 1.084497488253324, 0.6616259348165169, 30.189865905893072], [0.1023930236697197, 0.07322667698235097, 0.4063947510138716, 1.0966199078441043, 0.6607170907011506, 30.52732562881973], [0.1065988838672638, 0.06951195009544042, 0.4111004370170547, 1.117051806795602, 0.6633653927636083, 31.09610176360077], [0.10572896897792816, 0.06835445223858572, 0.41259720323148086, 1.1132755123704126, 0.6641905963041802, 30.990978585767266], [0.10537360608577728, 0.07077213154564777, 0.41170232881199936, 1.111736672463417, 0.6624669832585127, 30.948140892784085], [0.10712811350822449, 0.06923373560970414, 0.4118381696086163, 1.1209696047556377, 0.6635637404772865, 31.205164067885455], [0.10733970254659653, 0.0691712998125521, 0.4138843360342946, 1.1199289978245903, 0.6636082503390233, 31.17619601212752]]

In [ ]:
#Results from FL trial 2: Train on all homes
train_results:  [0.06898596219780932, 0.06548122778359505, 0.06185041400298901, 0.06054192711494171, 0.058454206690380324, 0.05690618117988075, 0.05569481511845242, 0.05503546991090291, 0.05418193751508395, 0.053413227141336866, 0.05262128071217649, 0.05209992935248824, 0.05133464310447817, 0.05037318027168617, 0.049760406571789456, 0.04947327085773233, 0.04883004258806736, 0.04831138391435008, 0.04784004755478911, 0.04739499937394071]
test_results:  [[0.07388950139284134, -0.6394206307101029, 0.2940040695688336, 0.8651581647483642, 0.7761451711873405, 42.73750960843977], [0.10229679197072983, -0.7187000854334691, 0.24852498062349612, 0.8859254704806616, 0.8136781573856805, 43.76338321680318], [0.10142817348241806, -0.7256445803898879, 0.24620718310013454, 0.8862384540920745, 0.8169658736995279, 43.778844135563546], [0.0964757427573204, -0.7036218577431197, 0.2571345084097417, 0.8821222316460292, 0.8065397265810448, 43.57550894957533], [0.0948667898774147, -0.6870919402250779, 0.2653881911042843, 0.8791050393278182, 0.7987140305459689, 43.42646419574404], [0.09232829511165619, -0.6723862043143909, 0.27381300604860054, 0.8772141657551705, 0.7917519420748073, 43.33305789066353], [0.09167767316102982, -0.6674416250445798, 0.27728940833314025, 0.8774828827796103, 0.7894110554511302, 43.34633210673394], [0.09230783581733704, -0.6718295387860991, 0.27591257752931375, 0.8783017496029307, 0.7914884009073095, 43.386782893833505], [0.09227342158555984, -0.6668164809479504, 0.2786549967785973, 0.8774768593896604, 0.7891150873202154, 43.34603456034722], [0.09188926219940186, -0.663334135803274, 0.2804593641079765, 0.8768874209479345, 0.7874664545742958, 43.31691718956662], [0.09317739307880402, -0.6676371394797516, 0.27811129764069026, 0.8775280676216676, 0.7895036083130942, 43.34856416984131], [0.093377485871315, -0.6678664609591507, 0.2778367616500098, 0.877078930313777, 0.78961217790754, 43.32637746364862], [0.09336987882852554, -0.6644323439395239, 0.2792824927301477, 0.8762281124679917, 0.7879863704906633, 43.284348344187144], [0.09260615706443787, -0.6560269490716168, 0.28308320393602676, 0.874632299913617, 0.7840070291709628, 43.20551760877393], [0.0935792624950409, -0.6595903949395858, 0.2810017749483793, 0.8750087359706437, 0.7856940695125773, 43.224112982729416], [0.09312581270933151, -0.6540594785849432, 0.2834550706028201, 0.8736186854571278, 0.7830755917253327, 43.155446582066276], [0.09380140155553818, -0.6580750915440914, 0.28175009703612847, 0.874551030793283, 0.784976680562069, 43.20150303669605], [0.09393875300884247, -0.6532669189158098, 0.28402766426613224, 0.8738440282240405, 0.7827003698615169, 43.16657817517668], [0.09293350577354431, -0.6430557061244582, 0.2887735304023372, 0.8723430365864373, 0.7778661096898173, 43.09243145016379], [0.09316611289978027, -0.6414575532448334, 0.28935691922657975, 0.8721005390241325, 0.7771095046804066, 43.08045243601204]]

In [ ]:
#Global training on 33 buildings, kmeans group 0
[0.055916693061590195, -0.08682051436345928, 0.2662437579818409, 0.9251781331394637, 0.7748182519969681, 25.754788813324193]
#Fed training on 33 buildings, kmeans group 0
[0.10733970254659653, 0.0691712998125521, 0.4138843360342946, 1.1199289978245903, 0.6636082503390233, 31.17619601212752]

In [ ]:
#Global training on 11 buildings, kmeans group 0
[0.10680125653743744, -0.2791650140765447, 0.17327582767193095, 0.9721052782034753, 0.9043568620623283, 29.816883628987515]

In [ ]:
#Box-Cox Global training on 11 buildings, kmeans group 0
[0.07675079256296158, -0.1594563360666088, 0.24777122017698072, 0.9572877984815846, 0.8197240244830643, 29.36239471873389]

In [ ]:
homes_aug = load_all_houses_with_device(r"C:\Users\aar245\Desktop\privacy_preserving_nn\input\1min_real_aug2019.csv", 'drye1')

In [ ]:
homes_aug.loc[homes_aug.dataid==2358]

In [ ]:
#results for test building 145 through 1222 with 6/10 trials done
{'Train_home_2602_Test_home_145_trial_0': [0.040176667273044586, 0.5350352393891653, 0.6995605416768369, 0.6202009862456976, 0.16763660105062883, 25.120263894399006], 'Train_home_2602_Test_home_145_trial_1': [0.05921102687716484, -2.0440782008511387, 0.7124453406067465, 0.6474282140808139, 0.24652965752880682, 26.279175281524658], 'Train_home_2602_Test_home_145_trial_2': [0.03823353722691536, 0.5057120170055666, 0.7560540177590247, 0.5195950316365615, 0.14908920077524376, 22.51904609468248], 'Train_home_2602_Test_home_145_trial_3': [0.05927851051092148, 0.30235999898475624, 0.6887774979077629, 0.6375544077244527, 0.25398679080020226, 25.147226519054836], 'Train_home_2602_Test_home_145_trial_4': [0.041999612003564835, 0.4882201798661358, 0.6926759932829819, 0.6551516045754432, 0.1793288404278918, 25.931260691748726], 'Train_home_2602_Test_home_145_trial_5': [0.03647385537624359, 0.47454400621218595, 0.7559350868535851, 0.5200739818305863, 0.14116170384252258, 22.709971851772732], 'Train_home_2602_Test_home_145_trial_6': [0.11966244131326675, -1.064580240229691, 0.495144746087016, 1.2211936160376278, 0.4605021188671462, 53.628720707363556], 'Train_home_2602_Test_home_145_trial_7': [0.1058136448264122, -1.0395888561208113, 0.5233829748688715, 1.1160730460487083, 0.4145315960798125, 48.14635025130378], 'Train_home_2602_Test_home_145_trial_8': [0.03477741405367851, 0.454301144430442, 0.7551700010595161, 0.472974838218259, 0.143859846549889, 19.323349661297268], 'Train_home_2602_Test_home_145_trial_9': [0.04593951627612114, -0.4233190250154665, 0.6992317961832466, 0.6502480240201282, 0.18772461796476841, 26.892483287387424], 'Train_home_1240_Test_home_518_trial_0': [0.16696804761886597, -0.8029177145635302, 0.330246179185051, 1.061495864107348, 0.5406991734830829, 55.39654350280762], 'Train_home_1240_Test_home_518_trial_1': [0.1421043872833252, -1.0309879809684908, 0.3603602061554123, 1.1030694507951269, 0.5076501240101836, 52.1834230211046], 'Train_home_1240_Test_home_518_trial_2': [0.1428082138299942, -0.7208309734200375, 0.342751374536081, 1.1026372274300804, 0.522375694525131, 50.94359147813585], 'Train_home_1240_Test_home_518_trial_3': [0.15462778508663177, -0.6270411330811336, 0.34384646018458653, 1.0691058538056373, 0.5142801474616763, 54.324424997965494], 'Train_home_1240_Test_home_518_trial_4': [0.1594161093235016, -1.0189699473034592, 0.33037198747186325, 1.0733883577778536, 0.5316598258287396, 54.39286524454753], 'Train_home_1240_Test_home_518_trial_5': [0.14113833010196686, -0.8375110719493595, 0.3455177502273711, 1.137111852290704, 0.5245533921351407, 51.706502956814234], 'Train_home_1240_Test_home_518_trial_6': [0.14234046638011932, -0.6232183947079312, 0.3522625589613987, 1.0677044460454845, 0.5106981537086099, 50.29233741760254], 'Train_home_1240_Test_home_518_trial_7': [0.16293306648731232, -1.0306040477256764, 0.36236931577444914, 1.0089313390172745, 0.516691203916662, 53.76832110087077], 'Train_home_1240_Test_home_518_trial_8': [0.1363573968410492, -0.7060437572134186, 0.3433265095072676, 1.151195724294903, 0.5232934690939703, 50.695475006103514], 'Train_home_1240_Test_home_518_trial_9': [0.15628387033939362, -0.8211889401026671, 0.3363625411426606, 1.0681309550050495, 0.5289489288953005, 53.33491698371039], 'Train_home_4735_Test_home_558_trial_0': [0.12675102055072784, -0.11930339027325093, 0.4802788458210964, 0.944528932682456, 0.4833046093056032, 41.863205973307295], 'Train_home_4735_Test_home_558_trial_1': [0.12433425337076187, -0.10112702446732241, 0.5012986948129039, 0.8827606380338984, 0.48203296433857873, 38.48077074686686], 'Train_home_4735_Test_home_558_trial_2': [0.11468223482370377, -0.12194030253824091, 0.4889351703058594, 0.9711741131216777, 0.476870979572468, 39.47108472188314], 'Train_home_4735_Test_home_558_trial_3': [0.129390150308609, -0.1359004096951919, 0.5049839859378912, 0.8891788854149493, 0.4638564648151501, 41.917315038045246], 'Train_home_4735_Test_home_558_trial_4': [0.140033558011055, -0.1706410681326712, 0.4856474780004399, 0.8698543070109417, 0.5003473818040641, 41.14278895060222], 'Train_home_4735_Test_home_558_trial_5': [0.09900928288698196, -0.05346552448121979, 0.5111732869945806, 0.9204994315260336, 0.4469891661558323, 34.45792821248372], 'Train_home_4735_Test_home_558_trial_6': [0.1286795437335968, -0.14580961247247468, 0.4924472289237465, 0.9102656020124285, 0.47861912364904097, 41.35941073099772], 'Train_home_4735_Test_home_558_trial_7': [0.13680428266525269, -0.14896007221441182, 0.47014193254964703, 0.8994016563156133, 0.5077707994950913, 40.9517396291097], 'Train_home_4735_Test_home_558_trial_8': [0.09889096021652222, -0.008830908738437527, 0.5201491769256953, 0.8616636255480336, 0.44119162476673657, 32.6402774810791], 'Train_home_4735_Test_home_558_trial_9': [0.12643541395664215, -0.11086637113714279, 0.48580187386427515, 0.9123752363222237, 0.47418812241451824, 41.112918090820315], 'Train_home_1417_Test_home_690_trial_0': [0.019165588542819023, 0.6104856993942045, 0.659494315811179, 0.5123483240655574, 0.11330509590610131, 14.646192193031311], 'Train_home_1417_Test_home_690_trial_1': [0.038431935012340546, -0.3641241625972258, 0.6376970088898113, 0.7559297975721981, 0.2771077171126618, 17.717881321907043], 'Train_home_1417_Test_home_690_trial_2': [0.035066064447164536, 0.6246295572866176, 0.7046141511977373, 0.5329441143393426, 0.17112835571118493, 18.455823212862015], 'Train_home_1417_Test_home_690_trial_3': [0.04504918307065964, 0.6571699933824947, 0.6457885051518524, 0.6987258420163697, 0.22070083845056906, 24.10329532623291], 'Train_home_1417_Test_home_690_trial_4': [0.03545080125331879, 0.649567985416053, 0.7002473555642348, 0.5813226098557667, 0.16894589076927927, 20.61495304107666], 'Train_home_1417_Test_home_690_trial_5': [0.02715524472296238, 0.7093453236217487, 0.7458484474693083, 0.502803935588508, 0.13946502288304521, 16.54526662826538], 'Train_home_1417_Test_home_690_trial_6': [0.020976660773158073, 0.7231491730458934, 0.6940137110823132, 0.5059721393508319, 0.12004722395466756, 14.941614329814911], 'Train_home_1417_Test_home_690_trial_7': [0.04401381313800812, -0.23111311875568147, 0.7114290870810238, 0.7064905761246116, 0.22941499466380932, 22.906581580638885], 'Train_home_1417_Test_home_690_trial_8': [0.03177583962678909, 0.009508680342778741, 0.6810942695538185, 0.6300617989866455, 0.1858192764845124, 18.20858120918274], 'Train_home_1417_Test_home_690_trial_9': [0.06696628034114838, 0.3869597120553992, 0.6467221488288177, 0.8414672779826585, 0.37823476636807246, 25.177850782871246], 'Train_home_6868_Test_home_914_trial_0': [0.06426986306905746, 0.27059763980017365, 0.7214155365880961, 0.5674727578389072, 0.24117406636847974, 25.556920906854053], 'Train_home_6868_Test_home_914_trial_1': [0.05524551123380661, 0.3896988501932942, 0.6825560517728253, 0.6323498420845219, 0.20034019150570678, 29.469518419296023], 'Train_home_6868_Test_home_914_trial_2': [0.11004830151796341, -0.37914493706580854, 0.5136280734377812, 1.0618399716067999, 0.38990874591967434, 50.648451123918804], 'Train_home_6868_Test_home_914_trial_3': [0.07309284806251526, 0.2652196324437294, 0.672405572077398, 0.5982670451249826, 0.251457248776689, 29.38952396029518], 'Train_home_6868_Test_home_914_trial_4': [0.059073884040117264, 0.2486139255608043, 0.6848645053935399, 0.6479025340384593, 0.2336971515836993, 27.678240503583634], 'Train_home_6868_Test_home_914_trial_5': [0.12356363236904144, -0.9855835689967737, 0.4753318323101653, 1.2644952079283454, 0.4703685443608513, 56.13791662549216], 'Train_home_6868_Test_home_914_trial_6': [0.07400612533092499, 0.24566078165494695, 0.6284446746126974, 0.7689593759946023, 0.27950413180641354, 34.40880254715208], 'Train_home_6868_Test_home_914_trial_7': [0.06604988873004913, 0.2573574601613081, 0.6429867838359319, 0.7031425751793077, 0.23529966576922126, 33.35652778262184], 'Train_home_6868_Test_home_914_trial_8': [0.08203399926424026, -0.032200636487886666, 0.6150767520358214, 0.8303072907515743, 0.31168039715766716, 36.932607484242276], 'Train_home_6868_Test_home_914_trial_9': [0.047203827649354935, 0.42000588523737675, 0.7309441313527008, 0.4909148439129791, 0.17855798757046015, 21.93263522405473], 'Train_home_2602_Test_home_950_trial_0': [0.11927787959575653, 0.046352587674025655, 0.41754165255971576, 1.2827117870857443, 0.5438604133509721, 47.5431809425354], 'Train_home_2602_Test_home_950_trial_1': [0.1528531312942505, -0.12381728627936998, 0.4773757168762001, 1.167701157922694, 0.5933252478622649, 50.83934307098389], 'Train_home_2602_Test_home_950_trial_2': [0.13351531326770782, -0.17123930113951757, 0.4600082401343012, 1.2782711296070417, 0.588600632248767, 49.0027060508728], 'Train_home_2602_Test_home_950_trial_3': [0.14812250435352325, -0.13462020781523398, 0.37216672278963575, 1.144665278326408, 0.5843766460066064, 49.033560276031494], 'Train_home_2602_Test_home_950_trial_4': [0.12520718574523926, 0.08346193317081785, 0.4976359594875329, 1.147793908627099, 0.4970672503167678, 48.86126780509949], 'Train_home_2602_Test_home_950_trial_5': [0.13294504582881927, -0.018841707046933165, 0.4806386463569075, 1.0024644723351939, 0.4845877159964664, 46.4788544178009], 'Train_home_2602_Test_home_950_trial_6': [0.17622457444667816, -0.171863538135422, 0.5337338574096545, 1.1200644271229074, 0.6159976597777657, 54.15232753753662], 'Train_home_2602_Test_home_950_trial_7': [0.1415855884552002, -0.023914679939418468, 0.4518631342544024, 1.2389841065103326, 0.5046261960875048, 58.749167919158936], 'Train_home_2602_Test_home_950_trial_8': [0.1284128725528717, -0.013981808404577703, 0.4411456490646878, 1.1002892819521393, 0.5152482726525561, 46.34315872192383], 'Train_home_2602_Test_home_950_trial_9': [0.11963028460741043, 0.06484867847716143, 0.47592350625407265, 1.0639966815793362, 0.45577128895140623, 47.19773769378662]}

In [ ]:
#results for test building 1222 thru end
{'Train_home_2602_Test_home_1222_trial_0': [0.09906239807605743, -0.24100798097998796, 0.5229885162715611, 1.1428107937884233, 0.5039197801549766, 37.967190424601235], 'Train_home_2602_Test_home_1222_trial_1': [0.11193954944610596, -1.113720777569146, 0.48617999930986894, 1.3962239630080935, 0.6288462981579928, 42.00305023899785], 'Train_home_2602_Test_home_1222_trial_2': [0.09310683608055115, -0.8831631021470309, 0.5141724408914005, 1.1981492723802556, 0.48135313091510235, 39.16655554594817], 'Train_home_2602_Test_home_1222_trial_3': [0.10624466091394424, -2.0068414234846603, 0.4995003038867523, 1.2413566242010952, 0.538577470582748, 41.384928385416664], 'Train_home_2602_Test_home_1222_trial_4': [0.08546129614114761, -0.615840368009092, 0.48594434747445775, 1.2578414604201493, 0.43178088557787836, 42.07447345168502], 'Train_home_2602_Test_home_1222_trial_5': [0.09245969355106354, -0.35806246333641384, 0.5490135716034724, 1.0117112756091171, 0.4261247728933431, 37.09872417096739], 'Train_home_2602_Test_home_1222_trial_6': [0.11427279561758041, -1.4600448269437858, 0.4767608466518975, 1.3851360989066548, 0.60022247157453, 44.566621957001864], 'Train_home_2602_Test_home_1222_trial_7': [0.08118017762899399, -0.547787734239102, 0.4830990463223475, 1.2068403110257535, 0.4542504707488536, 36.44946158373797], 'Train_home_2602_Test_home_1222_trial_8': [0.08306696265935898, -0.49346011491257424, 0.50939064649106, 1.0124033840786695, 0.4254974950745316, 33.40193162141023], 'Train_home_2602_Test_home_1222_trial_9': [0.1576751172542572, -1.0278210832184154, 0.32009510585006207, 1.9937383257611476, 0.8130953810081494, 65.33960999382867], 'Train_home_6564_Test_home_1240_trial_0': [0.11917684227228165, -0.4954280256634407, 0.24748814620549936, 1.4031000799475226, 0.6758871283377295, 41.81124136182997], 'Train_home_6564_Test_home_1240_trial_1': [0.14965501427650452, -1.4154715485553728, 0.3485154881453461, 1.56315865789403, 0.8402873478606415, 47.04930337270101], 'Train_home_6564_Test_home_1240_trial_2': [0.1419462114572525, -0.6415813522261136, 0.31813622384655815, 1.7276628257546207, 0.6930360458938227, 59.801720513237846], 'Train_home_6564_Test_home_1240_trial_3': [0.1854940503835678, -3.7517075486709723, 0.318381888480695, 1.4467436170631975, 0.9226975076810844, 49.15287494659424], 'Train_home_6564_Test_home_1240_trial_4': [0.155636265873909, -0.4423912868750493, 0.3487577417547374, 1.496145135719671, 0.6283401279976703, 62.62913513183594], 'Train_home_6564_Test_home_1240_trial_5': [0.17196276783943176, -3.376993930941349, 0.2932380146673061, 1.4910257437096281, 0.8876698648846316, 48.81516806284586], 'Train_home_6564_Test_home_1240_trial_6': [0.1663823276758194, -1.4505881713823947, 0.43896708166125437, 1.5300930302019675, 0.8156158315097823, 52.750437312655976], 'Train_home_6564_Test_home_1240_trial_7': [0.1724431812763214, -3.363182330459586, 0.2795568444952114, 1.291421833799408, 0.8081968388161485, 46.567582077450226], 'Train_home_6564_Test_home_1240_trial_8': [0.135407954454422, -0.7257749740121401, 0.34474361920387625, 1.7949339836528895, 0.7116858716153981, 57.715304692586265], 'Train_home_6564_Test_home_1240_trial_9': [0.12931455671787262, -0.32289173381463754, 0.34622796791720606, 1.3525514522262638, 0.6669569382315061, 44.31901762220595], 'Train_home_6564_Test_home_1417_trial_0': [0.10290171205997467, -1.5029603480093672, 0.6483833660917904, 0.8117420800798352, 0.3489185552846667, 40.45789533853531], 'Train_home_6564_Test_home_1417_trial_1': [0.07465717941522598, -1.4526493462050447, 0.5816737248916052, 0.8498151823819738, 0.3030329902174015, 35.38285452127457], 'Train_home_6564_Test_home_1417_trial_2': [0.08346616476774216, 0.053639163430157365, 0.6396339276405452, 0.7434465881872049, 0.2995335319516526, 35.01075604557991], 'Train_home_6564_Test_home_1417_trial_3': [0.07464534044265747, 0.3120495734326398, 0.5472677885458131, 1.0444356047734318, 0.36624372005228545, 35.97500675916672], 'Train_home_6564_Test_home_1417_trial_4': [0.12709377706050873, -2.227075723177148, 0.5928776641332472, 1.1375359784312935, 0.5991974734233126, 40.77614077925682], 'Train_home_6564_Test_home_1417_trial_5': [0.08021712303161621, -1.452228702740836, 0.6213616564571698, 0.8752946363184456, 0.32708307999153546, 36.27855461835861], 'Train_home_6564_Test_home_1417_trial_6': [0.07654384523630142, 0.09247021594779253, 0.6436909180452643, 0.7858861256038352, 0.3174487795196207, 32.02454096078873], 'Train_home_6564_Test_home_1417_trial_7': [0.06872062385082245, 0.43068486956878543, 0.6615720437803603, 0.7217241423933717, 0.2840174625019709, 29.512091130018234], 'Train_home_6564_Test_home_1417_trial_8': [0.06753621250391006, -1.6073392480548363, 0.6126103403584593, 0.7832514140235372, 0.3102525968768542, 28.814373195171356], 'Train_home_6564_Test_home_1417_trial_9': [0.06695272773504257, 0.4098260785306949, 0.6640038380422062, 0.7484666438170645, 0.2845935760526071, 29.75790375471115], 'Train_home_1222_Test_home_2318_trial_0': [0.09997638314962387, 0.054632621081276186, 0.5659553802031221, 0.7530568744640781, 0.3501922360386081, 36.33334544125725], 'Train_home_1222_Test_home_2318_trial_1': [0.08103517442941666, 0.25132167809242884, 0.6216788356761503, 0.6898205885158434, 0.2932244344271993, 32.217829059152045], 'Train_home_1222_Test_home_2318_trial_2': [0.08661296218633652, -0.10959338497693347, 0.7013250509034116, 0.5861818168483609, 0.2838386128108514, 30.229465007781982], 'Train_home_1222_Test_home_2318_trial_3': [0.13796520233154297, -0.7700398789807373, 0.2997347059101964, 1.206080518622651, 0.5486142918618192, 51.258446973912854], 'Train_home_1222_Test_home_2318_trial_4': [0.08022147417068481, 0.30364426938176425, 0.6332706142485122, 0.6230568731527457, 0.29222671768787, 28.90580721462474], 'Train_home_1222_Test_home_2318_trial_5': [0.07152999192476273, 0.1918049626558701, 0.6343089132750068, 0.6390013633083511, 0.264358768537845, 29.220183540793027], 'Train_home_1222_Test_home_2318_trial_6': [0.08860918134450912, 0.01751692995810382, 0.5802085904885219, 0.7134127917578813, 0.29466143568146674, 36.25626443414127], 'Train_home_1222_Test_home_2318_trial_7': [0.0864693820476532, 0.16811091512252888, 0.6026270892030838, 0.7276409987843071, 0.35224217287818654, 30.18734373765833], 'Train_home_1222_Test_home_2318_trial_8': [0.08432725816965103, 0.24971845687112793, 0.6032420659811235, 0.688037888309314, 0.32389601217448244, 30.273417164297665], 'Train_home_1222_Test_home_2318_trial_9': [0.07887700200080872, 0.10204223006495604, 0.5618520972973684, 0.7491082925147379, 0.2788860951455995, 35.80592155456543], 'Train_home_5192_Test_home_2358_trial_0': [0.08658791333436966, -0.2619182579564897, 0.4783228352853373, 1.2011430061751187, 0.4284650027425633, 41.02261479695638], 'Train_home_5192_Test_home_2358_trial_1': [0.0777062177658081, -1.2469803774923687, 0.5617799855099694, 1.1946611731678394, 0.43569187770900064, 36.00872802734375], 'Train_home_5192_Test_home_2358_trial_2': [0.07014928013086319, -0.49732618262417533, 0.47842721445624775, 1.3914755982945715, 0.43901337956740094, 37.57576052347819], 'Train_home_5192_Test_home_2358_trial_3': [0.058772388845682144, -0.5386391977754993, 0.4980663353022285, 1.3356899474680666, 0.393798250999174, 33.68929640452067], 'Train_home_5192_Test_home_2358_trial_4': [0.07453611493110657, -0.24406913427582033, 0.6317323740250061, 0.9334482353232384, 0.3899937808444991, 30.149909337361652], 'Train_home_5192_Test_home_2358_trial_5': [0.08047938346862793, -0.11019905600317749, 0.490692683519893, 1.156884952215367, 0.41943783572476256, 37.514045079549156], 'Train_home_5192_Test_home_2358_trial_6': [0.07132060080766678, -0.27695674131603054, 0.49799787532370837, 1.1930857250304636, 0.40136626810837595, 35.82881609598795], 'Train_home_5192_Test_home_2358_trial_7': [0.07003666460514069, -0.3144061584668663, 0.5101128382096966, 1.1760351745918736, 0.40091819577261634, 34.71975644429525], 'Train_home_5192_Test_home_2358_trial_8': [0.070203498005867, -0.2833082104280839, 0.5120197706905476, 1.1573080209341664, 0.3940674175316954, 34.843668619791664], 'Train_home_5192_Test_home_2358_trial_9': [0.06498127430677414, -0.24675486017570125, 0.5025277148792231, 1.265869492898613, 0.3867620947695472, 35.943467458089195], 'Train_home_10983_Test_home_2448_trial_0': [0.07022134214639664, 0.5420596972939747, 0.6508582716610677, 0.7822533842373959, 0.26756305588744295, 34.695818066596985], 'Train_home_10983_Test_home_2448_trial_1': [0.03564563766121864, 0.7534086271058809, 0.7385220003554361, 0.5407602892978411, 0.1335271857916637, 24.396538376808167], 'Train_home_10983_Test_home_2448_trial_2': [0.029757816344499588, 0.7603784933473792, 0.7324359015572989, 0.5600645390554294, 0.11373009267821181, 24.765690565109253], 'Train_home_10983_Test_home_2448_trial_3': [0.04853034391999245, 0.5586583744169257, 0.7426281739243419, 0.5868852907367123, 0.1980053818287014, 24.309517443180084], 'Train_home_10983_Test_home_2448_trial_4': [0.04722093045711517, 0.62822445773579, 0.6695897804842039, 0.6935397755605941, 0.19311121646317023, 28.66058623790741], 'Train_home_10983_Test_home_2448_trial_5': [0.032409023493528366, 0.7599143264628102, 0.6587317555666885, 0.6677408783306303, 0.13832344543719408, 26.440204620361328], 'Train_home_10983_Test_home_2448_trial_6': [0.16635377705097198, -0.24406992487323692, 0.3702073858185835, 1.6652088238421245, 0.6934151068185365, 67.51414442062378], 'Train_home_10983_Test_home_2448_trial_7': [0.07006754726171494, 0.4675709671548381, 0.6924800848965507, 0.7287044986595458, 0.2682128515143871, 32.17180836200714], 'Train_home_10983_Test_home_2448_trial_8': [0.04610772058367729, 0.6869441543341148, 0.7361879147918221, 0.5762737924247565, 0.18437829417452067, 24.35451191663742], 'Train_home_10983_Test_home_2448_trial_9': [0.050767723470926285, 0.6769906941954348, 0.7509243961788139, 0.5542500197774743, 0.19375247364810116, 24.543294489383698], 'Train_home_10983_Test_home_3488_trial_0': [0.06027887016534805, 0.17728971329444443, 0.6887566283489213, 0.6215338518819657, 0.22878403680847847, 27.675206650627985], 'Train_home_10983_Test_home_3488_trial_1': [0.05380718410015106, 0.2639087774244669, 0.7078769129720798, 0.5694127781555711, 0.19639283134829966, 26.365044678582084], 'Train_home_10983_Test_home_3488_trial_2': [0.059725698083639145, 0.025761634257935404, 0.6667080622724674, 0.6341981062668721, 0.2315796089289343, 27.642197460598418], 'Train_home_10983_Test_home_3488_trial_3': [0.06008531525731087, 0.02682466123793673, 0.6889073594966829, 0.6027638745658364, 0.2232256295213104, 27.419416173299155], 'Train_home_10983_Test_home_3488_trial_4': [0.05533034726977348, 0.3214800159448681, 0.6997858275229071, 0.624114034865055, 0.20585581326800975, 28.34985925886366], 'Train_home_10983_Test_home_3488_trial_5': [0.06839794665575027, 0.016808246430367976, 0.698704630825907, 0.6624675291094296, 0.2658128984531311, 28.808346441056994], 'Train_home_10983_Test_home_3488_trial_6': [0.05077262595295906, 0.44624831482532457, 0.6595801085134039, 0.6413420887962757, 0.19018364465805354, 28.93562469482422], 'Train_home_10983_Test_home_3488_trial_7': [0.06365349143743515, 0.14602583634882443, 0.6841464734984731, 0.6439807351374544, 0.2428113696859242, 28.53072075313992], 'Train_home_10983_Test_home_3488_trial_8': [0.05749800056219101, 0.1941494527857974, 0.6807680748685819, 0.6453839896537397, 0.21747890598599945, 28.83636186387804], 'Train_home_10983_Test_home_3488_trial_9': [0.05933461710810661, 0.022278753774078355, 0.637861101205981, 0.7381955269489242, 0.23136439545322857, 31.994085905287], 'Train_home_3488_Test_home_3996_trial_0': [0.11575361341238022, -0.22502604102250684, 0.4296522913746713, 0.9693566348904439, 0.47885777136598706, 39.60028562740404], 'Train_home_3488_Test_home_3996_trial_1': [0.09607679396867752, -0.03795137941436585, 0.4846073391567936, 0.9778831631931663, 0.431766793955034, 36.774174748634806], 'Train_home_3488_Test_home_3996_trial_2': [0.09058652073144913, -0.07202836162563477, 0.4438478727144029, 1.0824754699429995, 0.46785072345742124, 35.42101696559361], 'Train_home_3488_Test_home_3996_trial_3': [0.10216927528381348, -0.14616719571596345, 0.41908923894903394, 0.9420336823168087, 0.47044480439601166, 34.57520712638388], 'Train_home_3488_Test_home_3996_trial_4': [0.09267144650220871, 0.011216129341403798, 0.49249082284539614, 0.8610016907072982, 0.42152601507133985, 31.98985413142613], 'Train_home_3488_Test_home_3996_trial_5': [0.10041536390781403, -0.04692868958490146, 0.4516486691661037, 0.9267865504161905, 0.43294127610298316, 36.32767484625992], 'Train_home_3488_Test_home_3996_trial_6': [0.08911094814538956, -0.05693688565606732, 0.5091052456627033, 0.9271101491546239, 0.3999328382190963, 34.91097812263333], 'Train_home_3488_Test_home_3996_trial_7': [0.08994857221841812, -0.00986166057840682, 0.47814698075810225, 0.8893763363847286, 0.4302373058412482, 31.423784742549973], 'Train_home_3488_Test_home_3996_trial_8': [0.08606432378292084, -0.18558840910099425, 0.46548898190190224, 1.0238201887182818, 0.4368933032133179, 34.08459717886789], 'Train_home_3488_Test_home_3996_trial_9': [0.12056294828653336, -0.11692695110157482, 0.40125371686275296, 1.0379960000244726, 0.5054598430134668, 41.84172163204271], 'Train_home_6564_Test_home_4283_trial_0': [0.028009910136461258, 0.7026551922838807, 0.763412110502766, 0.3850485285601968, 0.08678269167442723, 21.00297286635951], 'Train_home_6564_Test_home_4283_trial_1': [0.03176438808441162, 0.45587475945364264, 0.7784476662763697, 0.3960633512591818, 0.10229586866642117, 20.784221937781886], 'Train_home_6564_Test_home_4283_trial_2': [0.029869332909584045, 0.6638267233727633, 0.7468093185458642, 0.4564992863715461, 0.09838480775762733, 23.422014637997275], 'Train_home_6564_Test_home_4283_trial_3': [0.026042049750685692, 0.6825552918235298, 0.7730070032195813, 0.3759005811971987, 0.08176485552559687, 20.233367104279367], 'Train_home_6564_Test_home_4283_trial_4': [0.027608593925833702, 0.5761140245042423, 0.7653534979592472, 0.38375609730288585, 0.08721142228576216, 20.531131581256265], 'Train_home_6564_Test_home_4283_trial_5': [0.02819415181875229, 0.5291803296452723, 0.7678294096714045, 0.3961223599280669, 0.08925048958597807, 21.147764977655914], 'Train_home_6564_Test_home_4283_trial_6': [0.026220371946692467, 0.6855135595590393, 0.7580062461192129, 0.36418923340617576, 0.0810735379979998, 19.90551879531459], 'Train_home_6564_Test_home_4283_trial_7': [0.02500646933913231, 0.7028706913410565, 0.775223692808435, 0.35964950918828514, 0.07844915379284621, 19.374489370145294], 'Train_home_6564_Test_home_4283_trial_8': [0.023368313908576965, 0.5946901198842942, 0.7785706596961235, 0.3443022339906561, 0.07179235268365992, 18.939813425666408], 'Train_home_6564_Test_home_4283_trial_9': [0.15557292103767395, -0.8501245933737515, 0.466248640811501, 1.0804045361154044, 0.47656706572384067, 59.6050446911862], 'Train_home_9002_Test_home_4414_trial_0': [0.06727452576160431, 0.3722280446049982, 0.616175590492429, 0.717870580943604, 0.28643595542569295, 28.494166082265426], 'Train_home_9002_Test_home_4414_trial_1': [0.0948755070567131, 0.2260529026788685, 0.4697632432510104, 0.852323984955402, 0.38880419781540465, 35.149129614538076], 'Train_home_9002_Test_home_4414_trial_2': [0.09543691575527191, 0.051549584850532876, 0.46915101839402124, 0.9797744024989794, 0.3976838050989534, 39.73665365880849], 'Train_home_9002_Test_home_4414_trial_3': [0.07743199914693832, 0.09166799089535776, 0.5709842634881974, 0.7732831423407533, 0.32823064805623187, 30.82950634859046], 'Train_home_9002_Test_home_4414_trial_4': [0.07215101271867752, 0.35755380771655937, 0.5231197246421465, 0.788916051143339, 0.31694497386887144, 30.351212871317962], 'Train_home_9002_Test_home_4414_trial_5': [0.0842733159661293, 0.1968619378275753, 0.5453874855557251, 0.8462484593068745, 0.342859612051917, 35.15267193074129], 'Train_home_9002_Test_home_4414_trial_6': [0.07307938486337662, 0.27386888984668534, 0.562018842059484, 0.7897621062124711, 0.3141660065658672, 31.04693506201919], 'Train_home_9002_Test_home_4414_trial_7': [0.06769915670156479, 0.11235411910519537, 0.5735075411878885, 0.7949668327930315, 0.3258219322273485, 27.915066504965022], 'Train_home_9002_Test_home_4414_trial_8': [0.13264892995357513, -0.18414329250033215, 0.35077422770494027, 1.0991116900411333, 0.5461836068192728, 45.11216704699458], 'Train_home_9002_Test_home_4414_trial_9': [0.10710166394710541, 0.050233580094908074, 0.4184178755366508, 1.0362076778247171, 0.4492378325316748, 41.7497041663345], 'Train_home_7935_Test_home_5587_trial_0': [0.05806110426783562, 0.47198973815917605, 0.6826879617495338, 0.6105704706278842, 0.21700354849688985, 27.608379811048508], 'Train_home_7935_Test_home_5587_trial_1': [0.048992279917001724, 0.47235897627994117, 0.6904619874987754, 0.5997319373974499, 0.19479780161190086, 25.49103730916977], 'Train_home_7935_Test_home_5587_trial_2': [0.0621502548456192, 0.2501665829983938, 0.6945880728066577, 0.6411271101531254, 0.23517236622251222, 28.634359061717987], 'Train_home_7935_Test_home_5587_trial_3': [0.1301395297050476, -0.5873377472211747, 0.44390297430094033, 1.205868327735263, 0.48044160733175495, 55.20205342769623], 'Train_home_7935_Test_home_5587_trial_4': [0.05410081148147583, 0.4562182450493112, 0.7006118391235748, 0.6330580047444723, 0.2064814863650707, 28.031921565532684], 'Train_home_7935_Test_home_5587_trial_5': [0.05965317413210869, 0.5162755606303098, 0.7304172843891479, 0.5707591347272299, 0.20855779356509857, 27.589684009552002], 'Train_home_7935_Test_home_5587_trial_6': [0.059780653566122055, 0.4411361384934333, 0.6805927776938809, 0.6073881982775157, 0.21675328473540129, 28.310529559850693], 'Train_home_7935_Test_home_5587_trial_7': [0.055954862385988235, 0.22736110819941735, 0.6466326482422597, 0.6538408417495574, 0.244247877469581, 25.314292654395103], 'Train_home_7935_Test_home_5587_trial_8': [0.04927569627761841, 0.582856138004595, 0.7391322024438703, 0.486069788813159, 0.16604770877889738, 24.377280443906784], 'Train_home_7935_Test_home_5587_trial_9': [0.058318935334682465, 0.32372355978352435, 0.6519054173933809, 0.669919830703861, 0.23818327243757098, 27.720934838056564], 'Train_home_9290_Test_home_5982_trial_0': [0.1561063528060913, -0.26960078432095513, 0.38855613329057154, 0.7790471964063976, 0.5772121446537969, 35.6070135307312], 'Train_home_9290_Test_home_5982_trial_1': [0.09761467576026917, 0.17660835248537163, 0.524469228043357, 0.6896694794241758, 0.3702034534575123, 30.73281557718913], 'Train_home_9290_Test_home_5982_trial_2': [0.07846306264400482, 0.31221611057654075, 0.5230546265681372, 0.6917820069341234, 0.33776945182013274, 27.15819111506144], 'Train_home_9290_Test_home_5982_trial_3': [0.09757433086633682, 0.14017686674611018, 0.5789274743628013, 0.6396630324606922, 0.3709946465819527, 28.431902551651], 'Train_home_9290_Test_home_5982_trial_4': [0.10278156399726868, 0.18417741855428676, 0.5492179225080243, 0.6773974082422313, 0.362048404107606, 32.49966030120849], 'Train_home_9290_Test_home_5982_trial_5': [0.12785866856575012, 0.015368791943528658, 0.4337747080813755, 0.7003361657843377, 0.47020484266165635, 32.18372603416443], 'Train_home_9290_Test_home_5982_trial_6': [0.1264028400182724, -0.0754608995856275, 0.42276725835578194, 0.8353473707831858, 0.5241095376428531, 34.0477601369222], 'Train_home_9290_Test_home_5982_trial_7': [0.08273441344499588, 0.1561343432226496, 0.5714908079238638, 0.6389933713978605, 0.3212670404388049, 27.8101362546285], 'Train_home_9290_Test_home_5982_trial_8': [0.09675367176532745, 0.167233976687957, 0.5395658032797216, 0.6614752186080357, 0.35594277611605296, 30.386982510884604], 'Train_home_9290_Test_home_5982_trial_9': [0.10652545094490051, -0.021836732639670314, 0.49043155870637906, 0.7633823028410914, 0.42084797370073085, 32.65554405212402], 'Train_home_4283_Test_home_5997_trial_0': [0.022164875641465187, 0.7922192699031295, 0.837987324942272, 0.25343496747266214, 0.07154488456768794, 13.269053152629308], 'Train_home_4283_Test_home_5997_trial_1': [0.02961716242134571, 0.7657498910050601, 0.8084482890670212, 0.3203689544028167, 0.09900677265839725, 16.1962936265128], 'Train_home_4283_Test_home_5997_trial_2': [0.02398291788995266, 0.7877499514528856, 0.7862947459543953, 0.3361124279089362, 0.07869816165974672, 17.310462701888312], 'Train_home_4283_Test_home_5997_trial_3': [0.0236910879611969, 0.8083597002208125, 0.7836732955052926, 0.3463697762266521, 0.08975545503390292, 15.450795287177677], 'Train_home_4283_Test_home_5997_trial_4': [0.03056757152080536, 0.7366453597958595, 0.7642182098588652, 0.38548842431941144, 0.10130361725641582, 19.657758372170584], 'Train_home_4283_Test_home_5997_trial_5': [0.03318268805742264, 0.6819466548506364, 0.7314139962958828, 0.3944427945170654, 0.11622189560874226, 19.032432260967436], 'Train_home_4283_Test_home_5997_trial_6': [0.033547647297382355, 0.6800365692785524, 0.6648409623026921, 0.5292577402025475, 0.13163563071324003, 22.795153776804607], 'Train_home_4283_Test_home_5997_trial_7': [0.04580925777554512, 0.6470119139511717, 0.770463120633304, 0.40336996695340616, 0.14765481643758802, 21.149296760559082], 'Train_home_4283_Test_home_5997_trial_8': [0.041734084486961365, 0.6734686177651741, 0.754424008976561, 0.3876689879137747, 0.13433587030033067, 20.35385431562151], 'Train_home_4283_Test_home_5997_trial_9': [0.03402641788125038, 0.7128901609231345, 0.771712070303887, 0.34774598319318767, 0.1272116788591592, 15.719475292024159], 'Train_home_914_Test_home_6069_trial_0': [0.14039751887321472, -0.43151422218692004, 0.2801285992957234, 1.5794423374872237, 0.6681475868216118, 56.08897249626391], 'Train_home_914_Test_home_6069_trial_1': [0.14417600631713867, -0.3887800113557237, 0.2692709681550513, 1.5333595154233657, 0.6678085149069145, 55.94634871049361], 'Train_home_914_Test_home_6069_trial_2': [0.060355015099048615, 0.06231736283878993, 0.6683461403983437, 0.7651799722055744, 0.29565582050745515, 26.398374701991226], 'Train_home_914_Test_home_6069_trial_3': [0.05570201203227043, 0.03853141289590484, 0.5999902794795771, 0.9060634258345002, 0.306798529407543, 27.801159945401277], 'Train_home_914_Test_home_6069_trial_4': [0.13770700991153717, -0.41976305404234204, 0.2606950185348158, 1.6375464259849064, 0.6934715908497704, 54.95506309740471], 'Train_home_914_Test_home_6069_trial_5': [0.13660623133182526, 0.02364835179303926, 0.34980598671830293, 1.2854226635260542, 0.5437217704246289, 54.5791103478634], 'Train_home_914_Test_home_6069_trial_6': [0.06318530440330505, 0.4566080306635347, 0.6512941894553534, 0.7793360752149195, 0.3077821948826269, 27.038591384887695], 'Train_home_914_Test_home_6069_trial_7': [0.06178698316216469, 0.009900009820975525, 0.632485308735618, 0.8336178337110135, 0.3074829421507249, 28.309332125114672], 'Train_home_914_Test_home_6069_trial_8': [0.14669638872146606, -0.3717767444046983, 0.29111517106138507, 1.4810333855778388, 0.6461899586345464, 56.821251609108664], 'Train_home_914_Test_home_6069_trial_9': [0.05925316363573074, 0.18400353695187968, 0.5807357551780199, 0.9088388492536097, 0.2826538285189958, 32.19810300884825], 'Train_home_7935_Test_home_6178_trial_0': [0.07355224341154099, -0.005695130393346022, 0.675476642268994, 0.7917165173116522, 0.3455320846633505, 28.481572735694147], 'Train_home_7935_Test_home_6178_trial_1': [0.10123283416032791, -0.3091872818313794, 0.45133091313284396, 1.3446001543175492, 0.45705457729762394, 50.3307250238234], 'Train_home_7935_Test_home_6178_trial_2': [0.08631636947393417, -0.02614491434183116, 0.6284440767616439, 0.8643709261611439, 0.3885866761875623, 32.44831214412566], 'Train_home_7935_Test_home_6178_trial_3': [0.06867983192205429, 0.14190710901233408, 0.6817126782347588, 0.6904888492007741, 0.3058028010996312, 26.207837781598492], 'Train_home_7935_Test_home_6178_trial_4': [0.13163480162620544, -0.8460219467429779, 0.376604748294334, 1.5175188077030128, 0.5956226656664637, 56.678758682743194], 'Train_home_7935_Test_home_6178_trial_5': [0.06899765878915787, 0.22019031280946233, 0.6429605213003505, 0.8057109440339928, 0.3047376029895167, 30.830053175649336], 'Train_home_7935_Test_home_6178_trial_6': [0.052074357867240906, 0.345079195814106, 0.6911744020429339, 0.6260946786253097, 0.2200884080941783, 25.035339201650313], 'Train_home_7935_Test_home_6178_trial_7': [0.10004402697086334, -0.281744415504879, 0.6237020489857541, 0.8959394865328104, 0.4623628414321621, 32.7622443168394], 'Train_home_7935_Test_home_6178_trial_8': [0.12689629197120667, -0.3223882981383162, 0.3883046120288521, 1.3330449639037916, 0.5423662098295863, 52.70936965942383], 'Train_home_7935_Test_home_6178_trial_9': [0.07776453346014023, 0.06920769622838428, 0.6626335029907444, 0.781075506935284, 0.34328364934348593, 29.902548297759026], 'Train_home_10089_Test_home_6594_trial_0': [0.1699872761964798, -1.175565531954119, 0.31830947403067, 1.0752249519344643, 0.6152611945266387, 50.204529309884094], 'Train_home_10089_Test_home_6594_trial_1': [0.19127629697322845, -1.72521237216394, 0.3339859314111374, 1.0516020998600768, 0.6181214927094366, 54.99528327355018], 'Train_home_10089_Test_home_6594_trial_2': [0.19114111363887787, -1.4224092573771863, 0.3460505666024509, 1.0480443768277594, 0.6146555490895161, 55.07933443020551], 'Train_home_10089_Test_home_6594_trial_3': [0.11501190811395645, -0.6994893867837226, 0.4424778157358807, 0.7889534535964573, 0.3880024819458858, 39.52267860754942], 'Train_home_10089_Test_home_6594_trial_4': [0.10255571454763412, -0.7010646050402826, 0.4625916843529695, 0.7493684010036828, 0.34498132766460093, 37.648386930808044], 'Train_home_10089_Test_home_6594_trial_5': [0.11062644422054291, -0.9375056371034194, 0.575775030146675, 0.6936077445344647, 0.3656452045349897, 35.46497901586386], 'Train_home_10089_Test_home_6594_trial_6': [0.12362848967313766, -0.42014876637283566, 0.44640983182075494, 0.8970394521705795, 0.39638751400843064, 47.282106986412636], 'Train_home_10089_Test_home_6594_trial_7': [0.148359477519989, -0.5311173203250121, 0.39150936251303836, 1.0245895002772056, 0.5095499690430261, 50.415621366256325], 'Train_home_10089_Test_home_6594_trial_8': [0.18882639706134796, -0.6755575678808453, 0.33705303009485504, 1.0426367938437966, 0.6082684292383491, 54.699977801396294], 'Train_home_10089_Test_home_6594_trial_9': [0.1888926923274994, -1.559972860415517, 0.3015522663226815, 1.052671570355938, 0.6298412157498834, 53.35359336168338], 'Train_home_5587_Test_home_6672_trial_0': [0.04506482928991318, 0.5338309068828723, 0.6580917829850188, 0.4464525710097193, 0.17684773112671764, 19.226501532963344], 'Train_home_5587_Test_home_6672_trial_1': [0.04378471523523331, 0.5445805128577751, 0.6369148095663871, 0.4783378566897804, 0.15404207560009311, 22.977596827915736], 'Train_home_5587_Test_home_6672_trial_2': [0.037277672439813614, 0.6597212027042452, 0.7310576055100712, 0.3846196062777027, 0.1406332027252016, 17.229752972012474], 'Train_home_5587_Test_home_6672_trial_3': [0.021934768185019493, 0.7031837320954716, 0.717289979072026, 0.3731467451291949, 0.08427678512694382, 16.413131100790842], 'Train_home_5587_Test_home_6672_trial_4': [0.026403110474348068, 0.6820221445171308, 0.7096058743319904, 0.41401011702101664, 0.09474843878232503, 19.497579006921676], 'Train_home_5587_Test_home_6672_trial_5': [0.029187366366386414, 0.7019956595414455, 0.7007273979241203, 0.42497769144618114, 0.12390350464491454, 16.91858904702323], 'Train_home_5587_Test_home_6672_trial_6': [0.049780599772930145, 0.533581507067253, 0.6443135730596393, 0.5061190315943203, 0.18057958691442075, 23.5792954989842], 'Train_home_5587_Test_home_6672_trial_7': [0.04464860260486603, 0.5995675092495647, 0.6702200681351708, 0.5595482400722018, 0.19336106821624108, 21.835496130443754], 'Train_home_5587_Test_home_6672_trial_8': [0.03613051027059555, 0.5969326484132665, 0.6699356455904306, 0.4954578574881681, 0.13278496791992375, 22.783401739029657], 'Train_home_5587_Test_home_6672_trial_9': [0.03518660366535187, 0.616210079123636, 0.6447002565125295, 0.5242679855746385, 0.1411554876358763, 22.086131164005824], 'Train_home_2602_Test_home_6703_trial_0': [0.16244640946388245, -0.9070079244970037, 0.33504990139192053, 1.8023062901254618, 0.7504671811224515, 65.93160803921252], 'Train_home_2602_Test_home_6703_trial_1': [0.06730378419160843, 0.08090593704065219, 0.5171710436649477, 1.0079244150844189, 0.34120493795654977, 33.599966439856104], 'Train_home_2602_Test_home_6703_trial_2': [0.06149892508983612, 0.11663674098452463, 0.530492455075616, 0.9753156465950718, 0.31417274309685445, 32.264942674751744], 'Train_home_2602_Test_home_6703_trial_3': [0.0494186207652092, 0.3816104047416584, 0.5306873211154927, 0.882906734246153, 0.25427776500635024, 28.999049933559924], 'Train_home_2602_Test_home_6703_trial_4': [0.06661251932382584, 0.13247635311081904, 0.4856958650785102, 1.0836459029754029, 0.33313324735657734, 36.61946195579437], 'Train_home_2602_Test_home_6703_trial_5': [0.06254266202449799, 0.19054668225061866, 0.5377281614535238, 0.9764454171144888, 0.33614817478400677, 30.7029611058982], 'Train_home_2602_Test_home_6703_trial_6': [0.07441992312669754, 0.06793930311387329, 0.5126763166871386, 1.0368778263148937, 0.35508189198795675, 36.726110872015894], 'Train_home_2602_Test_home_6703_trial_7': [0.060796745121479034, 0.1334032415324993, 0.5342304261551077, 0.9823405699830766, 0.3160498613776901, 31.935483472893036], 'Train_home_2602_Test_home_6703_trial_8': [0.13294759392738342, -0.6971624962946411, 0.3470785124678825, 1.833945907490076, 0.6924795586974029, 59.50407823309841], 'Train_home_2602_Test_home_6703_trial_9': [0.0655357837677002, 0.22977506795988006, 0.5271338983321312, 0.9637645191393569, 0.310268974639501, 34.40311739818159], 'Train_home_8327_Test_home_6706_trial_0': [0.08798699080944061, -0.18177299450114326, 0.5623418013350074, 0.587286235666233, 0.2457869163616669, 35.53008300154956], 'Train_home_8327_Test_home_6706_trial_1': [0.08498114347457886, -0.205486842682318, 0.5871572228052836, 0.5711777184415354, 0.23832834340117037, 34.41952076242931], 'Train_home_8327_Test_home_6706_trial_2': [0.09818774461746216, -0.24371075006094192, 0.5596780831114981, 0.6118840068157498, 0.26314136972196217, 38.58548365066301], 'Train_home_8327_Test_home_6706_trial_3': [0.0777105987071991, 0.04353752959407342, 0.6275780589036271, 0.5345581199025586, 0.21264666115846717, 33.01439219802173], 'Train_home_8327_Test_home_6706_trial_4': [0.07762718945741653, -0.05480536003202109, 0.5564260796987093, 0.596285863828873, 0.22801119332994024, 34.3082747103563], 'Train_home_8327_Test_home_6706_trial_5': [0.07438323646783829, -0.010101076865358067, 0.6054685061604284, 0.540810421040424, 0.20878537840782635, 32.56167435290209], 'Train_home_8327_Test_home_6706_trial_6': [0.07326830923557281, -0.04297048529595057, 0.638093317452617, 0.5392800274921036, 0.20867953915883014, 31.99906850928691], 'Train_home_8327_Test_home_6706_trial_7': [0.07763416320085526, -0.07805425344960001, 0.609806115515373, 0.5357955021580011, 0.21726441971536498, 32.35564275286091], 'Train_home_8327_Test_home_6706_trial_8': [0.08195451647043228, -0.1491123020688649, 0.5832652627780754, 0.5871933761461194, 0.2378421773867657, 34.19415503829273], 'Train_home_8327_Test_home_6706_trial_9': [0.09002068638801575, -0.3177760091178559, 0.5210563824134385, 0.6281109444914609, 0.25353617314272486, 37.689938189378424], 'Train_home_6564_Test_home_6907_trial_0': [0.09081520140171051, -0.08471449856067317, 0.6311851908683008, 0.7829323585645511, 0.39246195308645504, 30.617656707763672], 'Train_home_6564_Test_home_6907_trial_1': [0.10315681993961334, -0.26467097409412566, 0.5060585259983142, 1.1367338385602606, 0.41716036729299366, 47.505116406609034], 'Train_home_6564_Test_home_6907_trial_2': [0.10493253916501999, -0.9326077728941364, 0.6883482331870499, 0.962428225314318, 0.49842414314143724, 34.24255898419548], 'Train_home_6564_Test_home_6907_trial_3': [0.09611622244119644, -0.4046209776683135, 0.6533183374496695, 0.897317160754111, 0.4390926011913418, 33.1950465370627], 'Train_home_6564_Test_home_6907_trial_4': [0.10162191838026047, -0.6618503920570648, 0.6330124590396405, 1.056325279530716, 0.5202109666507589, 34.873237385469324], 'Train_home_6564_Test_home_6907_trial_5': [0.1032496765255928, -0.7015677024794357, 0.6589714108180554, 0.9120865914823072, 0.49792802508699086, 31.962806757758646], 'Train_home_6564_Test_home_6907_trial_6': [0.17032909393310547, -1.6947111664161998, 0.5953548760534827, 1.277581226144375, 0.7248399499988767, 50.736670550178076], 'Train_home_6564_Test_home_6907_trial_7': [0.12963742017745972, -0.6826622096628038, 0.5912520619118898, 0.9887309294019916, 0.5361029809409741, 40.40610072192024], 'Train_home_6564_Test_home_6907_trial_8': [0.09797833859920502, -0.18023498904950097, 0.5354930589105799, 0.7395749846727493, 0.439652192711963, 27.854140954859115], 'Train_home_6564_Test_home_6907_trial_9': [0.09398982673883438, -0.32744901534331394, 0.4900633054544937, 1.2562775664326318, 0.4328444878723629, 46.102165446561926], 'Train_home_8849_Test_home_7069_trial_0': [0.02715522237122059, 0.764742672745967, 0.7620002494960105, 0.3819318871267532, 0.12446735725538428, 14.08220010054739], 'Train_home_8849_Test_home_7069_trial_1': [0.015977874398231506, 0.7896432121515872, 0.7604213950657331, 0.287175879416331, 0.07354822539983949, 10.543419085050884], 'Train_home_8849_Test_home_7069_trial_2': [0.022174395620822906, 0.793521575431435, 0.7980940494508117, 0.3058583109707619, 0.1056288627358398, 10.851159986696745], 'Train_home_8849_Test_home_7069_trial_3': [0.01771567389369011, 0.7772824923604802, 0.7476816296552132, 0.34519835202430316, 0.08757078647475684, 11.801951935416774], 'Train_home_8849_Test_home_7069_trial_4': [0.020348116755485535, 0.7136387909540785, 0.7253299522690692, 0.31039610982271953, 0.0974227568336046, 10.956372411627518], 'Train_home_8849_Test_home_7069_trial_5': [0.06284583359956741, 0.4715354328066808, 0.5457060986635639, 0.7099142666854521, 0.2937319105431508, 25.669531270077353], 'Train_home_8849_Test_home_7069_trial_6': [0.023435348644852638, 0.786145484978176, 0.7873124243016723, 0.34109617713511553, 0.10698195207323484, 12.627704695651406], 'Train_home_8849_Test_home_7069_trial_7': [0.0198507197201252, 0.7452840747464979, 0.7387778853929332, 0.33297691618014913, 0.09474184814974476, 11.790581703186035], 'Train_home_8849_Test_home_7069_trial_8': [0.026894455775618553, 0.7205137323034605, 0.7198945882518849, 0.38559548828691564, 0.11872730402133827, 14.761511074869256], 'Train_home_8849_Test_home_7069_trial_9': [0.03482116386294365, 0.6705011983044855, 0.6621293800227834, 0.5165415527820777, 0.1593847503031346, 19.07165943948846], 'Train_home_1706_Test_home_7159_trial_0': [0.09108869731426239, 0.2961086982752785, 0.4524663898562966, 0.912440702233858, 0.3887099847197201, 36.13517400953505], 'Train_home_1706_Test_home_7159_trial_1': [0.0882933959364891, 0.34115654805763246, 0.6029594155154644, 0.6973743092010237, 0.31714080041200726, 32.81170336405436], 'Train_home_1706_Test_home_7159_trial_2': [0.09782489389181137, 0.25843887495359275, 0.5718017319857687, 0.6875960373431487, 0.3175686654505411, 35.795779652065704], 'Train_home_1706_Test_home_7159_trial_3': [0.1486900895833969, -0.24229432030418963, 0.4392791564946824, 0.8486983677909476, 0.48261271342215156, 44.18993462456597], 'Train_home_1706_Test_home_7159_trial_4': [0.19684213399887085, -1.2322623703581774, 0.3256155432640703, 1.155461943626007, 0.5917985420455554, 64.95109939575195], 'Train_home_1706_Test_home_7159_trial_5': [0.09823668748140335, 0.23303961458481245, 0.5574579802617672, 0.8257213112564591, 0.3986884459040834, 34.384306801689995], 'Train_home_1706_Test_home_7159_trial_6': [0.0883835181593895, 0.1475385525910191, 0.47907362080174815, 0.9052876464401435, 0.38226929178863744, 35.373263888888886], 'Train_home_1706_Test_home_7159_trial_7': [0.10207685083150864, 0.34298028649644796, 0.5698693946127623, 0.6700186444358001, 0.32423658929568167, 35.64829847547743], 'Train_home_1706_Test_home_7159_trial_8': [0.09460662305355072, 0.43143979416776723, 0.6468292585748084, 0.6567841103778046, 0.30395552837381784, 34.547833336724175], 'Train_home_1706_Test_home_7159_trial_9': [0.12482301890850067, 0.10826014348532166, 0.43549100625875525, 0.795653220128391, 0.48475821042214673, 34.62422614627414], 'Train_home_6178_Test_home_8162_trial_0': [0.09550192207098007, 0.0377112825181859, 0.5701126500565369, 0.7035436114280388, 0.352548694096512, 32.20852140946822], 'Train_home_6178_Test_home_8162_trial_1': [0.07898425310850143, -1.1012275871864685, 0.6226075683223828, 0.6818326191408132, 0.3123568958801791, 29.137606360695578], 'Train_home_6178_Test_home_8162_trial_2': [0.06543022394180298, 0.40955557948483584, 0.6755750336787149, 0.555514471065215, 0.23265858025210076, 26.402282281355426], 'Train_home_6178_Test_home_8162_trial_3': [0.07967749983072281, 0.35476462370226963, 0.6694632319143854, 0.5994624939448703, 0.25696222404993974, 31.413417209278453], 'Train_home_6178_Test_home_8162_trial_4': [0.12569423019886017, -0.6865417744739757, 0.4446745251282361, 1.0775223962039644, 0.5063604324920042, 45.20314138585871], 'Train_home_6178_Test_home_8162_trial_5': [0.06542861461639404, -1.678834914490271, 0.6086302530866827, 0.6243980151085432, 0.25003942810689106, 27.612609646537088], 'Train_home_6178_Test_home_8162_trial_6': [0.07874567806720734, -0.9900774208240624, 0.608403951815984, 0.7091596462664587, 0.2860249166194633, 32.995412761514835], 'Train_home_6178_Test_home_8162_trial_7': [0.05705755949020386, 0.4910375074516443, 0.7199461369882532, 0.48654814967753096, 0.1878147458927698, 24.980208266865123], 'Train_home_6178_Test_home_8162_trial_8': [0.0649183839559555, -2.011976772560246, 0.5989952573511044, 0.6622807273036794, 0.2735788636272908, 26.559137387709185], 'Train_home_6178_Test_home_8162_trial_9': [0.06982077658176422, -1.6245951932024427, 0.6339346611255432, 0.5686128624498599, 0.24666629999791415, 27.20058618892323], 'Train_home_558_Test_home_8327_trial_0': [0.0734190046787262, -0.18557633031766388, 0.6742010768987615, 0.616330804648298, 0.26011028206486386, 29.400283658227256], 'Train_home_558_Test_home_8327_trial_1': [0.050764091312885284, 0.5471018069124145, 0.6826633826490021, 0.5669883172886101, 0.2015265723630562, 24.137102138164433], 'Train_home_558_Test_home_8327_trial_2': [0.03778652101755142, 0.7095179210069718, 0.758994706345793, 0.4506444257925929, 0.1537882748384416, 18.71260988989542], 'Train_home_558_Test_home_8327_trial_3': [0.09503957629203796, 0.24676724358747235, 0.5925258923463197, 0.6982039167489463, 0.3397166586040113, 33.010844319365745], 'Train_home_558_Test_home_8327_trial_4': [0.0415339320898056, 0.5823884519729895, 0.657580442037221, 0.5388231741419823, 0.16856625204263034, 22.43703043738077], 'Train_home_558_Test_home_8327_trial_5': [0.05777944624423981, 0.5011621322418828, 0.6671652289355806, 0.5294907016662918, 0.22444715386687736, 23.03585209957389], 'Train_home_558_Test_home_8327_trial_6': [0.06065402179956436, 0.5188057535864092, 0.6610296176609902, 0.5735656114316318, 0.2331237643057556, 25.21987176495929], 'Train_home_558_Test_home_8327_trial_7': [0.04788490757346153, 0.061447224822643844, 0.7550155667993332, 0.4861393850362596, 0.17527042098121368, 22.445927575577137], 'Train_home_558_Test_home_8327_trial_8': [0.045644428580999374, 0.04147574993918129, 0.6475875978750657, 0.6371479493789712, 0.19989433754462105, 24.58749394084132], 'Train_home_558_Test_home_8327_trial_9': [0.08436752110719681, 0.37801867048512383, 0.6044100618973848, 0.7145025302223279, 0.31248839405597967, 32.60106953909231], 'Train_home_145_Test_home_9002_trial_0': [0.1146293357014656, -0.27440093185765213, 0.3040767862530386, 1.5377410029388336, 0.6062463644858428, 49.137887954711914], 'Train_home_145_Test_home_9002_trial_1': [0.10528463125228882, -0.2838121895207724, 0.3153178442255147, 1.5469267407232137, 0.5797258999973781, 47.47868765554121], 'Train_home_145_Test_home_9002_trial_2': [0.0812605768442154, 0.17867648441966918, 0.4057358304513441, 0.9499220338541461, 0.4505686236155204, 28.953003145033314], 'Train_home_145_Test_home_9002_trial_3': [0.0726332813501358, -0.054438988934819105, 0.4020492161759309, 1.1721970543438114, 0.47623225411910813, 30.21371841430664], 'Train_home_145_Test_home_9002_trial_4': [0.08997683972120285, -0.10323975726500141, 0.3348941421224701, 1.184949677510878, 0.5259213478745769, 34.26072566739975], 'Train_home_145_Test_home_9002_trial_5': [0.07194391638040543, 0.0567148756872715, 0.448660746336439, 0.9571825252673398, 0.41788236806433043, 27.849762239763812], 'Train_home_145_Test_home_9002_trial_6': [0.08545844256877899, 0.0023511538415550426, 0.37297624174541105, 1.2057670070407318, 0.46515016434053374, 37.43793745963804], 'Train_home_145_Test_home_9002_trial_7': [0.08019377291202545, 0.12603548182361723, 0.3955413682666241, 1.0461922624995397, 0.47728335305025027, 29.707259547325872], 'Train_home_145_Test_home_9002_trial_8': [0.07394816726446152, 0.1624942704101791, 0.3970135503863338, 1.084201266947889, 0.4755822812120458, 28.49038807038338], 'Train_home_145_Test_home_9002_trial_9': [0.09187285602092743, -0.0019929105444094174, 0.34409399973843674, 1.193553405033958, 0.5181549813800198, 35.76482717452511], 'Train_home_950_Test_home_9290_trial_0': [0.1576719433069229, -0.42530434753619833, 0.35869170653504057, 1.1917755712105398, 0.5547272029421517, 57.247455465382544], 'Train_home_950_Test_home_9290_trial_1': [0.10155104100704193, -0.35423895301482355, 0.6481167405678193, 0.7336064077959067, 0.3757300156080131, 33.50875604563746], 'Train_home_950_Test_home_9290_trial_2': [0.12472879141569138, -0.017216942909693382, 0.45214542039986777, 1.0251248719678645, 0.4471603305281462, 48.32445164384513], 'Train_home_950_Test_home_9290_trial_3': [0.1456962525844574, -0.4677398041048449, 0.3380620289686621, 1.3203484497874494, 0.5866601963632359, 55.41623582511112], 'Train_home_950_Test_home_9290_trial_4': [0.14954429864883423, -0.20642840673558369, 0.3310621372824766, 1.2959124691357657, 0.587819526269772, 55.71706837621228], 'Train_home_950_Test_home_9290_trial_5': [0.1542765200138092, -0.5347290606039339, 0.3303762193870943, 1.3363366455783412, 0.6021989398524202, 57.85787437702047], 'Train_home_950_Test_home_9290_trial_6': [0.13884444534778595, -0.4221392197134675, 0.30353420085833044, 1.4051974561680798, 0.6177275832530446, 53.37718029679923], 'Train_home_950_Test_home_9290_trial_7': [0.15695449709892273, -0.4385148630191426, 0.32863305390737246, 1.2353801671694629, 0.5884440299692391, 55.687273880531045], 'Train_home_950_Test_home_9290_trial_8': [0.11406445503234863, -0.1533325702469134, 0.43384363882084837, 1.0790714605202034, 0.44024326125647, 47.24921180462015], 'Train_home_950_Test_home_9290_trial_9': [0.13740554451942444, -0.16808068715546895, 0.3150354653856135, 1.3075107976782245, 0.5965943343456825, 50.89288908859779], 'Train_home_6126_Test_home_984_trial_0': [0.14826421439647675, -0.17262388863607145, 0.2710553693075103, 1.1511370139604082, 0.6356091200473512, 45.379521818722], 'Train_home_6126_Test_home_984_trial_1': [0.13595008850097656, -0.2740116411880503, 0.35734198240687637, 1.1309058400228018, 0.5427353103073689, 47.87453455083511], 'Train_home_6126_Test_home_984_trial_2': [0.14046885073184967, 0.007511491017183745, 0.2923143752007899, 1.1835855094961967, 0.610562605290101, 46.01889195161707], 'Train_home_6126_Test_home_984_trial_3': [0.1325802505016327, -0.8920434746349253, 0.3443618318209501, 1.360887291896908, 0.5786133342755582, 52.69862545237822], 'Train_home_6126_Test_home_984_trial_4': [0.13604789972305298, -0.11224610033334667, 0.3025477894918969, 1.182626088415687, 0.6142923025212947, 44.264027707717], 'Train_home_6126_Test_home_984_trial_5': [0.12703101336956024, 0.009504869195241565, 0.36970937193274594, 1.1290591135016568, 0.5388765688863276, 44.98045287412756], 'Train_home_6126_Test_home_984_trial_6': [0.11921171098947525, -0.12248526947191186, 0.39309282258125217, 1.0730911581187363, 0.48872159484150557, 44.23649305455825], 'Train_home_6126_Test_home_984_trial_7': [0.16116970777511597, -1.3317579497763123, 0.3088419742638467, 1.3775439920005879, 0.637753368586313, 58.83325184092802], 'Train_home_6126_Test_home_984_trial_8': [0.13773411512374878, -0.15237639739344677, 0.3125431593850294, 1.1018295925826158, 0.5932428619348058, 43.23247859057258], 'Train_home_6126_Test_home_984_trial_9': [0.14184077084064484, 0.06675885409265903, 0.3593849997112926, 1.073917705828996, 0.5214835548973091, 49.3648879107307], 'Train_home_518_Test_home_3403_trial_0': [0.1084427684545517, 0.21233776275264588, 0.5964539868713494, 0.946570904442452, 0.49393885737689147, 35.12103319168091], 'Train_home_518_Test_home_3403_trial_1': [0.04886928200721741, 0.5909472780016262, 0.7188899575641375, 0.5418202895303076, 0.18415456522983, 24.29939115047455], 'Train_home_518_Test_home_3403_trial_2': [0.07067002356052399, 0.5028488252690099, 0.6655246985736559, 0.5967096436970691, 0.28033309331643386, 25.422053396701813], 'Train_home_518_Test_home_3403_trial_3': [0.07369441539049149, 0.4535070020430827, 0.669047496813405, 0.7548540856868967, 0.32160076249591063, 29.23258411884308], 'Train_home_518_Test_home_3403_trial_4': [0.07868828624486923, 0.503473447439232, 0.6326996769491727, 0.6329965539812052, 0.28885709436760887, 29.14171326160431], 'Train_home_518_Test_home_3403_trial_5': [0.06739223748445511, 0.4084477253062398, 0.578345707540268, 0.8915571703676021, 0.31775456916715444, 31.95611548423767], 'Train_home_518_Test_home_3403_trial_6': [0.05148507282137871, 0.6444653380084865, 0.7221612699003677, 0.599359815676439, 0.21689740245595246, 24.043699264526367], 'Train_home_518_Test_home_3403_trial_7': [0.09818534553050995, 0.3510779126607132, 0.6065790472240852, 0.8641507719804442, 0.35104110396932464, 40.84744358062744], 'Train_home_518_Test_home_3403_trial_8': [0.10018754005432129, 0.2199318170727168, 0.4662750747096973, 1.1804800069886532, 0.4684299509192102, 42.669190883636475], 'Train_home_518_Test_home_3403_trial_9': [0.09692121297121048, 0.24717714478809177, 0.6397946098716365, 0.7440516136298064, 0.3213101659325555, 37.93011426925659], 'Train_home_6706_Test_home_4735_trial_0': [0.04820743948221207, 0.06052624512502198, 0.7148865990742709, 0.7361432327039485, 0.25046410704477806, 23.945150511605398], 'Train_home_6706_Test_home_4735_trial_1': [0.07549852132797241, -0.5046713702694878, 0.52133068602794, 1.2449570954655802, 0.395369789301761, 40.17686244419643], 'Train_home_6706_Test_home_4735_trial_2': [0.046750668436288834, 0.19815151788387197, 0.7189308968410503, 0.6922069661378991, 0.23779353755682167, 22.99907772881644], 'Train_home_6706_Test_home_4735_trial_3': [0.0459446981549263, 0.21077565633658604, 0.7213924686013502, 0.7111344486415155, 0.23369964957847697, 23.62738813672747], 'Train_home_6706_Test_home_4735_trial_4': [0.043756283819675446, 0.22010976560075876, 0.7283385082180031, 0.6806188455151914, 0.23885736864211646, 21.07135382152739], 'Train_home_6706_Test_home_4735_trial_5': [0.04772942513227463, 0.03806626752296571, 0.7156054005975013, 0.7237551758445512, 0.25501796801343135, 22.892530872708274], 'Train_home_6706_Test_home_4735_trial_6': [0.062396466732025146, -0.05740649351409551, 0.6200322943216249, 0.9912005296285954, 0.30826583747402164, 33.90648969014486], 'Train_home_6706_Test_home_4735_trial_7': [0.056150536984205246, -0.07359175310708296, 0.6803129868132808, 0.8332577715214161, 0.2863367185698349, 27.614866029648553], 'Train_home_6706_Test_home_4735_trial_8': [0.040004126727581024, 0.07932380593175657, 0.7087229263390883, 0.6991669230833207, 0.21325529885955158, 22.165243761880056], 'Train_home_6706_Test_home_4735_trial_9': [0.05810766667127609, -0.1947991976314639, 0.593561102661418, 0.9970397425088137, 0.29119945449373574, 33.623431977771574], 'Train_home_6907_Test_home_5192_trial_0': [0.10184518992900848, -0.05951170102743667, 0.4931783498542924, 0.9720341674574954, 0.4187407385998339, 39.95430145263672], 'Train_home_6907_Test_home_5192_trial_1': [0.1127358078956604, -0.10166650532802149, 0.49460692886922564, 0.9253282172812353, 0.4302755876236725, 40.972992706298825], 'Train_home_6907_Test_home_5192_trial_2': [0.123016357421875, -0.11522887607497548, 0.4877985812113643, 0.9042421883368621, 0.4514271426089545, 41.64344482421875], 'Train_home_6907_Test_home_5192_trial_3': [0.1134433001279831, -0.16424181309065483, 0.5157169110018106, 0.86940097518486, 0.44168819796071795, 37.73721752166748], 'Train_home_6907_Test_home_5192_trial_4': [0.12473054230213165, -0.21946118618766158, 0.49970105908239243, 0.9237477946428998, 0.44146874637286404, 44.10755462646485], 'Train_home_6907_Test_home_5192_trial_5': [0.10063375532627106, -0.1604477269197496, 0.5154039533225337, 0.9305111255071221, 0.4200751755241708, 37.67254285812378], 'Train_home_6907_Test_home_5192_trial_6': [0.10880228132009506, -0.12729192263890787, 0.5026293017539162, 0.9106781886483571, 0.4257541829588244, 39.33061227798462], 'Train_home_6907_Test_home_5192_trial_7': [0.11208789795637131, -0.1088074606495932, 0.49660565094141174, 0.9001621989378932, 0.43015200958962224, 39.64096632003784], 'Train_home_6907_Test_home_5192_trial_8': [0.10897031426429749, -0.12667923203296513, 0.4953467915244635, 0.9648537127783108, 0.4281684814603034, 41.499386405944826], 'Train_home_6907_Test_home_5192_trial_9': [0.11103089153766632, -0.1284986872315108, 0.49928587206075736, 0.9072968258913177, 0.431675520868408, 39.43869419097901], 'Train_home_5587_Test_home_6126_trial_0': [0.06995668262243271, -0.662993334236359, 0.5294057722823269, 0.6361225156855035, 0.23399555882687706, 32.140236085461034], 'Train_home_5587_Test_home_6126_trial_1': [0.06963611394166946, -0.504081911045034, 0.5193866333992896, 0.6336304256422585, 0.2539403174602783, 29.364704562771706], 'Train_home_5587_Test_home_6126_trial_2': [0.044668421149253845, -0.050219546588806854, 0.7042956875734222, 0.4662167855261306, 0.16433511505722057, 21.416317970521987], 'Train_home_5587_Test_home_6126_trial_3': [0.05904698744416237, -0.04412061345945901, 0.6309074392250936, 0.5373695742312555, 0.20565115007908927, 26.0751217257592], 'Train_home_5587_Test_home_6126_trial_4': [0.04916989430785179, -0.26488876780753934, 0.5977477810646857, 0.5265022185427152, 0.18226107833605693, 24.00447674720518], 'Train_home_5587_Test_home_6126_trial_5': [0.06570622324943542, -0.19373801018281742, 0.6284645535391377, 0.5458097720324033, 0.22579314025292616, 26.842553923206946], 'Train_home_5587_Test_home_6126_trial_6': [0.06103118136525154, -0.2298164570765566, 0.5901192216574314, 0.5724879014713354, 0.21895106358901156, 26.968558557571903], 'Train_home_5587_Test_home_6126_trial_7': [0.09178046137094498, -0.2756947035629795, 0.5905427417057874, 0.6337808602124958, 0.31570600265053145, 31.138180302035423], 'Train_home_5587_Test_home_6126_trial_8': [0.05937225744128227, -0.14761079251256357, 0.6320043986127729, 0.5248986058786199, 0.19775573076419817, 26.632785366427512], 'Train_home_5587_Test_home_6126_trial_9': [0.04725729674100876, -0.24230529475333754, 0.7118292496678232, 0.4580450229691987, 0.16565826444664172, 22.082623943205803], 'Train_home_558_Test_home_6868_trial_0': [0.16719608008861542, -0.5868113672021118, 0.41032028760201417, 0.765200435622865, 0.44097812330133584, 49.03102156695198], 'Train_home_558_Test_home_6868_trial_1': [0.16250145435333252, -0.545761712233157, 0.4383082574560275, 0.7604133381054639, 0.415549244720986, 50.25406343796674], 'Train_home_558_Test_home_6868_trial_2': [0.14097605645656586, -0.3088846336665188, 0.5012292515329744, 0.6784613078073064, 0.34420845565576863, 46.96081576627844], 'Train_home_558_Test_home_6868_trial_3': [0.18068313598632812, -0.6504730549231655, 0.3996044398258919, 0.8068671793132502, 0.4873142867649282, 50.558852812823126], 'Train_home_558_Test_home_6868_trial_4': [0.16175490617752075, -0.5555928886697463, 0.4527871035853535, 0.7859521379731769, 0.44017713271986997, 48.81044387817383], 'Train_home_558_Test_home_6868_trial_5': [0.14537139236927032, -0.24627237259244303, 0.5454005393412734, 0.723060000289735, 0.38209629431119085, 46.490815330954156], 'Train_home_558_Test_home_6868_trial_6': [0.14499609172344208, -0.39115931703836515, 0.49554117176879087, 0.7253227112987676, 0.3797437416886853, 46.8040750727934], 'Train_home_558_Test_home_6868_trial_7': [0.15936458110809326, -0.5228810321269878, 0.4619363118162023, 0.7499356339765653, 0.4162842020342499, 48.51908066693474], 'Train_home_558_Test_home_6868_trial_8': [0.18462814390659332, -0.6851100583566799, 0.3711215265155913, 0.8098955951322443, 0.48080094118362604, 52.559154117808625], 'Train_home_558_Test_home_6868_trial_9': [0.18160974979400635, -0.668388147064257, 0.4182985366460068, 0.7978428579653547, 0.4662073959623413, 52.52476041457232], 'Train_home_5997_Test_home_8849_trial_0': [0.03122751973569393, -0.3221437902228144, 0.7612479958538596, 0.4920074204313253, 0.18470960397026892, 14.057444042629665], 'Train_home_5997_Test_home_8849_trial_1': [0.07387908548116684, -2.7373934931488764, 0.6272601958607609, 0.8481610326308404, 0.4509914263758666, 23.48109033372667], 'Train_home_5997_Test_home_8849_trial_2': [0.013577751815319061, 0.9024517071373231, 0.8251088013094752, 0.33556503734851395, 0.06604969670860117, 11.657903459337023], 'Train_home_5997_Test_home_8849_trial_3': [0.07858799397945404, -3.110466533145379, 0.7137291891524803, 0.8981178436826911, 0.5352925379059189, 22.283581256866455], 'Train_home_5997_Test_home_8849_trial_4': [0.01466638594865799, 0.8826394853042143, 0.819590724658815, 0.326756396088613, 0.07680449948790839, 10.545015864902073], 'Train_home_5997_Test_home_8849_trial_5': [0.013102342374622822, 0.8915796830734983, 0.8105746269126382, 0.35436516044935984, 0.07292201563064242, 10.760388586256239], 'Train_home_5997_Test_home_8849_trial_6': [0.014327176846563816, 0.7950847363411571, 0.6976007310822288, 0.33966320393805194, 0.0879141547457124, 9.35485389497545], 'Train_home_5997_Test_home_8849_trial_7': [0.017718646675348282, 0.7741659293446284, 0.7151858297525949, 0.4113066198608788, 0.10446715445440968, 11.789710998535156], 'Train_home_5997_Test_home_8849_trial_8': [0.014214381575584412, 0.8853322853601586, 0.8068338620778953, 0.3630089639255309, 0.07901588235916986, 11.036149978637695], 'Train_home_5997_Test_home_8849_trial_9': [0.06801040470600128, -2.2760326455527773, 0.8092551853920978, 0.714060285820826, 0.39419351362979904, 20.82032405005561], 'Train_home_8849_Test_home_1706_trial_0': [0.06765400618314743, 0.38442512076248536, 0.49194360184487085, 0.80252311676287, 0.42504796046039345, 21.587376356124878], 'Train_home_8849_Test_home_1706_trial_1': [0.050874754786491394, 0.37805430521976807, 0.5266752248645902, 0.6199591397844011, 0.3498834413122423, 15.234518885612488], 'Train_home_8849_Test_home_1706_trial_2': [0.13470710813999176, -0.03228802365662853, 0.44553973479988596, 0.7703424419447835, 0.49506776339545167, 35.42390012741089], 'Train_home_8849_Test_home_1706_trial_3': [0.06618659198284149, 0.21509746017690298, -2.3786158494970207, 0.8800217696357073, 0.4839412949633683, 20.34030294418335], 'Train_home_8849_Test_home_1706_trial_4': [0.07219929993152618, 0.23022902236505588, -0.4278930684435463, 0.64837183913404, 0.3500528114238507, 22.600095093250275], 'Train_home_8849_Test_home_1706_trial_5': [0.08315329253673553, 0.09062834123934477, -10.625740499743507, 0.681206819132139, 0.3785388453073494, 25.289176106452942], 'Train_home_8849_Test_home_1706_trial_6': [0.0735403299331665, 0.4418611809690804, 0.683785156907594, 0.5413481954706714, 0.2896857175326985, 23.225329399108887], 'Train_home_8849_Test_home_1706_trial_7': [0.09255551546812057, 0.1691686827853998, 0.5000424898613754, 0.6100604011774603, 0.3075474633653563, 31.027709484100342], 'Train_home_8849_Test_home_1706_trial_8': [0.12860816717147827, 0.08990707284122385, 0.5091464492813148, 0.6439560087530662, 0.36651065318129755, 38.18781661987305], 'Train_home_8849_Test_home_1706_trial_9': [0.05590525269508362, 0.26547195442986804, 0.3851598042170199, 0.773137446142178, 0.5366172946680533, 13.612297117710114], 'Train_home_8162_Test_home_2602_trial_0': [0.12527203559875488, -0.6379548360878275, 0.66086776974041, 0.6446243842256046, 0.36438973228453403, 37.452555656433105], 'Train_home_8162_Test_home_2602_trial_1': [0.13537265360355377, -0.22261713741299727, 0.645732178671518, 0.5774641300855612, 0.3437210180646981, 38.43585725625356], 'Train_home_8162_Test_home_2602_trial_2': [0.17137257754802704, -0.2723633741322111, 0.6222415324817638, 0.5983732661438325, 0.4016026074527063, 43.15227667490641], 'Train_home_8162_Test_home_2602_trial_3': [0.12046145647764206, -0.26618145395980336, 0.6153776750330243, 0.5914892923373088, 0.3370714028806126, 35.723977406819664], 'Train_home_8162_Test_home_2602_trial_4': [0.15047971904277802, -0.7956212760619784, 0.6778438101292132, 0.5560402295271503, 0.33255591970698417, 42.52126439412435], 'Train_home_8162_Test_home_2602_trial_5': [0.15464968979358673, -0.31424180498449933, 0.6269087577632705, 0.6553896944766997, 0.4021311971075552, 42.595885356267296], 'Train_home_8162_Test_home_2602_trial_6': [0.16449350118637085, -0.5710392085651662, 0.5984335995802581, 0.6856024339402943, 0.40067514327358955, 47.568050463994346], 'Train_home_8162_Test_home_2602_trial_7': [0.12845592200756073, -0.1529492218335428, 0.6218702413479278, 0.6567796601596338, 0.37343317161578243, 38.18103051185608], 'Train_home_8162_Test_home_2602_trial_8': [0.13785161077976227, -0.9394607786952132, 0.6465655142958316, 0.5996997901970118, 0.3600280630213953, 38.80574778715769], 'Train_home_8162_Test_home_2602_trial_9': [0.1700718104839325, -0.8711423232822454, 0.48437535312699553, 0.7633334929415495, 0.45619494967144253, 48.09311024347941], 'Train_home_6672_Test_home_6564_trial_0': [0.18061162531375885, -1.0904659793000762, 0.3299541550599873, 0.8232292944610097, 0.5052245345485914, 49.73576220344095], 'Train_home_6672_Test_home_6564_trial_1': [0.19414469599723816, -1.1782452378118573, 0.3215300264211887, 0.8544722310609679, 0.5515321857367552, 50.832251717062555], 'Train_home_6672_Test_home_6564_trial_2': [0.22527393698692322, -1.5945898351680232, 0.20602755764969938, 0.9245830148595457, 0.6192013489500325, 56.847539705388684], 'Train_home_6672_Test_home_6564_trial_3': [0.2159985452890396, -1.2583161893571355, 0.2699813725707755, 0.8682349941950136, 0.5736887500071137, 55.24570257523481], 'Train_home_6672_Test_home_6564_trial_4': [0.21184679865837097, -1.1693806435223255, 0.2938310617100885, 0.8495174529721091, 0.5621627054481858, 54.10268835460438], 'Train_home_6672_Test_home_6564_trial_5': [0.18048930168151855, -1.1937706236444914, 0.42995891418105664, 0.7560384606719239, 0.4625689597257035, 49.854639025295484], 'Train_home_6672_Test_home_6564_trial_6': [0.15248645842075348, -0.5001802084346972, 0.25097009133469145, 0.794762460237531, 0.45518829724536647, 44.994998174555164], 'Train_home_6672_Test_home_6564_trial_7': [0.17243480682373047, -0.9720377835602463, 0.37406272440038923, 0.8063251177802016, 0.49478474424445357, 47.49036483203663], 'Train_home_6672_Test_home_6564_trial_8': [0.14343386888504028, -0.7017231647733893, 0.4447891473962856, 0.7045811390772905, 0.3937178153717183, 43.37948239550871], 'Train_home_6672_Test_home_6564_trial_9': [0.16561417281627655, -0.746870729310829, 0.37357008254488633, 0.7944541738242151, 0.4774468695466247, 46.572331582798675], 'Train_home_558_Test_home_7935_trial_0': [0.19044987857341766, -0.39817969578507073, 0.3116279762702574, 0.9703738334649877, 0.5815468583177132, 53.70586734347873], 'Train_home_558_Test_home_7935_trial_1': [0.18760144710540771, -0.33266320507410224, 0.28501415995621227, 0.9544844160152041, 0.6134324077045417, 49.331574228074814], 'Train_home_558_Test_home_7935_trial_2': [0.2134706825017929, -0.49238779686514117, 0.2651871387873128, 0.9694480879547831, 0.6280054130584983, 55.69114006890191], 'Train_home_558_Test_home_7935_trial_3': [0.1700744479894638, -0.6005226442700737, 0.2912536099204162, 0.9586341755542278, 0.5803721328574895, 47.47578186459012], 'Train_home_558_Test_home_7935_trial_4': [0.18737219274044037, -0.4213834057347993, 0.3324143591163672, 0.9553103997028527, 0.5723128025282084, 52.85704220665826], 'Train_home_558_Test_home_7935_trial_5': [0.15535548329353333, -0.19630416740965917, 0.4321782375384741, 0.8049601296497219, 0.44766693291825155, 47.209849993387856], 'Train_home_558_Test_home_7935_trial_6': [0.20294511318206787, -0.5377956364890651, 0.2898768036232316, 0.9446524562481264, 0.6021610377706961, 53.80525620778402], 'Train_home_558_Test_home_7935_trial_7': [0.20361799001693726, -0.5280456246254803, 0.23887599189953604, 0.9569454814333834, 0.6286655353120286, 52.38058429294162], 'Train_home_558_Test_home_7935_trial_8': [0.15376727283000946, -0.24820415654139893, 0.4094528926370212, 0.9741390250110613, 0.5018446419032375, 50.44315603044298], 'Train_home_558_Test_home_7935_trial_9': [0.20251770317554474, -0.46329920882826, 0.25677813915802433, 0.9657082458233968, 0.6081956513238596, 54.3440883424547], 'Train_home_5192_Test_home_10089_trial_0': [0.10028188675642014, -0.367117948900562, 0.30314447389968935, 1.8704902124328318, 0.6627216338618703, 47.833647046770366], 'Train_home_5192_Test_home_10089_trial_1': [0.11402930319309235, -0.17886648194210528, 0.29290835979154933, 1.6291141774703646, 0.631768635067879, 49.69316537039621], 'Train_home_5192_Test_home_10089_trial_2': [0.11741386353969574, -0.1971696805702297, 0.34265306688712593, 1.4400819130670257, 0.5716889100384427, 49.9842905317034], 'Train_home_5192_Test_home_10089_trial_3': [0.10391214489936829, -0.8904512860017214, 0.433701588162155, 1.5280141615076388, 0.6732690569346966, 39.85581942967006], 'Train_home_5192_Test_home_10089_trial_4': [0.09458842873573303, -0.6930084962284734, 0.5321862316182644, 1.2435541831011212, 0.5319575047314535, 37.36908721923828], 'Train_home_5192_Test_home_10089_trial_5': [0.10159078240394592, 0.016292012025037255, 0.33580089388114515, 1.5305737884011905, 0.5506819355219559, 47.719342367989675], 'Train_home_5192_Test_home_10089_trial_6': [0.08218681812286377, -0.2005552534077177, 0.3571982220056034, 1.868225220799239, 0.5872069157758162, 44.19029562813895], 'Train_home_5192_Test_home_10089_trial_7': [0.099154993891716, -0.19778414701700583, 0.31392561878780695, 1.6995120096623848, 0.6146879292987802, 46.33091245378767], 'Train_home_5192_Test_home_10089_trial_8': [0.09655160456895828, -0.31090837782911557, 0.28492328474838885, 1.9482702569022625, 0.6742660133402051, 47.14809472220285], 'Train_home_5192_Test_home_10089_trial_9': [0.10495851933956146, -0.09188232304187083, 0.3252679547451966, 1.549821602591946, 0.5867930189624216, 46.849092211042134], 'Train_home_6907_Test_home_10182_trial_0': [0.29818105697631836, -0.9205206909283145, 0.34956575212939084, 0.8787952959460317, 0.5435487327265355, 81.47342173258464], 'Train_home_6907_Test_home_10182_trial_1': [0.24442686140537262, -0.4487722953442157, 0.3324374567459, 0.9552694748612919, 0.5479813156926058, 72.01048914591472], 'Train_home_6907_Test_home_10182_trial_2': [0.276532381772995, -0.85423722326862, 0.3363675836654964, 0.913699759466286, 0.5491451299363769, 77.7587178548177], 'Train_home_6907_Test_home_10182_trial_3': [0.2909637987613678, -0.6744408847674235, 0.35066276385569106, 0.8868696009302522, 0.5401358029512703, 80.7388432820638], 'Train_home_6907_Test_home_10182_trial_4': [0.21922951936721802, -0.31468424578043924, 0.3430576651284815, 0.9799291414369012, 0.5332389074680691, 68.08611551920573], 'Train_home_6907_Test_home_10182_trial_5': [0.28417903184890747, -0.6324499489685543, 0.3283108929433726, 0.9137126291909592, 0.5615404018755903, 78.14611053466797], 'Train_home_6907_Test_home_10182_trial_6': [0.2934662997722626, -0.8757166351264859, 0.34889978202554567, 0.8825158886934903, 0.5448264510917222, 80.33583323160808], 'Train_home_6907_Test_home_10182_trial_7': [0.23244953155517578, -0.37919577925246156, 0.3612218005467311, 0.9497590696899579, 0.5211293204922184, 71.59510167439778], 'Train_home_6907_Test_home_10182_trial_8': [0.2776681184768677, -0.8508055436314228, 0.3419140609368762, 0.9117217020485978, 0.5459067465506611, 78.37120946248372], 'Train_home_6907_Test_home_10182_trial_9': [0.2303399294614792, -0.39762697622905613, 0.32781770322635945, 0.9745775792558659, 0.5470369989959781, 69.35146458943684], 'Train_home_5192_Test_home_10983_trial_0': [0.09207908064126968, 0.3498927779613134, 0.674798388590823, 0.5851343116359345, 0.20548309577314816, 44.312590789794925], 'Train_home_5192_Test_home_10983_trial_1': [0.1513613611459732, -0.48384536233791253, 0.6239714135569071, 0.6530919106938389, 0.2947599397305542, 56.67709503173828], 'Train_home_5192_Test_home_10983_trial_2': [0.10639644414186478, 0.31701871250371666, 0.7088912692625428, 0.49053994008375956, 0.20780614668120484, 42.44531936645508], 'Train_home_5192_Test_home_10983_trial_3': [0.22653968632221222, -1.1259583536705258, 0.4280217939258149, 0.8519057188927861, 0.4543413014868552, 71.78609161376953], 'Train_home_5192_Test_home_10983_trial_4': [0.23762929439544678, -1.162426313882976, 0.39866254001108953, 0.8960947758468529, 0.4851282385623619, 74.17953186035156], 'Train_home_5192_Test_home_10983_trial_5': [0.18207159638404846, -0.3532625234800665, 0.38629050911681134, 1.0289092991580662, 0.5155278653391775, 61.412085723876956], 'Train_home_5192_Test_home_10983_trial_6': [0.2633078694343567, -0.7495307922073751, 0.400378731012224, 0.8468832481571459, 0.485085083842973, 77.68839721679687], 'Train_home_5192_Test_home_10983_trial_7': [0.2438463717699051, -1.2000700052140651, 0.39374686561633476, 0.8739494827312906, 0.4981165019569178, 72.30334854125977], 'Train_home_5192_Test_home_10983_trial_8': [0.18384647369384766, -0.48547799747215575, 0.4441785332687592, 0.9340582352649585, 0.4595319617976828, 63.15390625], 'Train_home_5192_Test_home_10983_trial_9': [0.07232393324375153, 0.082384482198875, 0.7005465628781312, 0.5642427046404825, 0.20552171223026944, 33.55652618408203]}

In [ ]:
#1 round of FL, trained on all data, tested on 2358
Round 0 results:  [0.07405087351799011, -1.2491973647905144, 0.7047115864778972, 0.8195495188152665, 0.48308521414335215, 21.23089901606242]

In [ ]:
#1 round of FL, trained on all data tested on 2358 retrial (second take)
[[0.06951961666345596, -1.0547907920686839, 0.7311992588450115, 0.7456100673854843, 0.45352463815322563, 19.315455238024395], [0.06750611960887909, -0.9061764415008356, 0.7431598779071023, 0.7048023497628592, 0.4403891934243316, 18.258307973543804], [0.04297441616654396, -0.6392584912691109, 0.8125841782201447, 0.4744438811163561, 0.2803518699064789, 12.290740092595419]]

Round 0 results:  [0.06951961666345596, -1.0547907920686839, 0.7311992588450115, 0.7456100673854843, 0.45352463815322563, 19.315455238024395]
Round 1 results:  [0.06750611960887909, -0.9061764415008356, 0.7431598779071023, 0.7048023497628592, 0.4403891934243316, 18.258307973543804]
Round 2 results:  [0.04297441616654396, -0.6392584912691109, 0.8125841782201447, 0.4744438811163561, 0.2803518699064789, 12.290740092595419]

In [ ]:
#all 20 rounds results for FL
FL_results = [[0.06951961666345596, -1.0547907920686839, 0.7311992588450115, 0.7456100673854843, 0.45352463815322563, 19.315455238024395],
 [0.06750611960887909, -0.9061764415008356, 0.7431598779071023, 0.7048023497628592, 0.4403891934243316, 18.258307973543804],
 [0.04297441616654396, -0.6392584912691109, 0.8125841782201447, 0.4744438811163561, 0.2803518699064789, 12.290740092595419],
 [0.07527746260166168, -1.1321102858024428, 0.7133773582568644, 0.7908635820821566, 0.49108701108305475, 20.487773418426514],
 [0.06078772619366646, -0.9909178343987756, 0.7560586497334231, 0.6352026388321399, 0.3965604598159795, 16.4552876551946],
 [0.06729450821876526, -1.1279164035403797, 0.7078740864442855, 0.7361251794856674, 0.4390087399870242, 19.069743792215984],
 [0.03205053508281708, 0.2828068573659976, 0.7832790662095058, 0.42367462842894427, 0.20908782968145428, 10.975533564885458],
 [0.03972587361931801, 0.00046629515384349896, 0.7347972250315615, 0.48810910100089355, 0.2591594291053503, 12.644745429356893],
 [0.03881887346506119, 0.10475708558699538, 0.7167735279518588, 0.49345413640767144, 0.2532424364276869, 12.783211628595987],
 [0.044760655611753464, -0.1677143401281589, 0.6904550401939484, 0.540701311319558, 0.2920047632228599, 14.007176717122396],
 [0.04850225895643234, 0.05459620717059697, 0.7148736055951348, 0.5115025474377614, 0.3164138529907328, 13.250766038894653],
 [0.04988951236009598, 0.030102795840318237, 0.715130458322368, 0.5244570155533159, 0.32546384098156134, 13.586358944574991],
 [0.0633954107761383, -0.047742737008218315, 0.6740045526960716, 0.6332575540069433, 0.41357218055692735, 16.40489912033081],
 [0.06450808048248291, -0.09526510258711278, 0.6491365343570882, 0.6276351688916061, 0.42083087357915144, 16.25924801826477],
 [0.06549400091171265, -0.19760707133527125, 0.6671337254158477, 0.6168143856732332, 0.4272627511257778, 15.97892944018046],
 [0.05474377050995827, -0.1290505136790722, 0.696677594833347, 0.5843581071749195, 0.3571315516785887, 15.138130982716879],
 [0.06714557856321335, -0.1691848869123519, 0.6660485223422357, 0.6555396596983776, 0.43803713171654163, 16.9821298122406],
 [0.055342722684144974, -0.014301236927656094, 0.6970973325733955, 0.5805985456244764, 0.3610389377675811, 15.040737390518188],
 [0.059734467417001724, -0.03992679593688886, 0.6858098351038175, 0.5994954767458252, 0.38968928660048263, 15.530273199081421],
 [0.08048170059919357, -1.4764057743724204, 0.45971914384046236, 1.727740589820461, 0.5250378801530454, 44.758108139038086]]

In [ ]:
Trial 2 finished with value: 0.048315126448869705 and parameters:
        {'hidden_size_1': 30,
         'hidden_size_2': 123,
         'fc1': 299, 'fc2': 18,
         'weight_decay': 0.005038971768580936,
         'learning_rate': 0.000624235019529808,
         'window_size': 280}.
        Best is trial 2 with value: 0.048315126448869705

In [ ]:
Trial 21 finished with value: 0.05254843831062317 and parameters:
        {'hidden_size_1': 114,
         'hidden_size_2': 83,
         'fc1': 182, 'fc2': 28,
         'weight_decay': 0.07786274942658475,
         'learning_rate': 0.000305222263105676,
         'window_size': 2}
        Best is trial 21 with value: 0.05254843831062317

In [ ]:
best trial:
[0.0510893389582634]
{'hidden_size_1': 91,
 'hidden_size_2': 138,
 'fc1': 381, 'fc2': 27,
 'weight_decay': 0.009457205008621654,
 'learning_rate': 0.0003242301834854019,
 'window_size': 141}

In [ ]:
best trial:
[0.037125106900930405]
{'hidden_size_1': 44,
 'hidden_size_2': 129,
 'fc1': 340, 'fc2': 22,
 'weight_decay': 0.021543979275305797,
 'learning_rate': 0.00022534369918008793,
 'window_size': 169}

In [ ]:
import numpy as np
y_sequence = []
def normalize_y(y_sequence):
    y_sequence = np.log(np.add(y_sequence, 1))
    try:
        return (y_sequence - np.min(y_sequence)) / (np.max(y_sequence) - np.min(y_sequence))
    except ValueError:  # raised if `y` is empty.
        return []

In [ ]:
normalize_y(y_sequence)

In [ ]:
homes_aug = load_all_houses_with_device(r"C:\Users\aar245\Desktop\privacy_preserving_nn\input\1min_real_aug2019.csv", 'drye1')

In [ ]:
homes_aug.loc[homes_aug.dataid == 2358]